 <font size="6">Базовые методы Классического Машинного Обучения</font>

# Разделение train-validation-test 

## Примеры ошибок в данных и при разбиении

Машинное обучение - это не только об алгоритмах, но и о данных. При работе с данными необходимо соблюдать некоторую культуру. В этой части лекции мы разберем ряд ошибок, допускаемых при работе с данными и их разбиением.

### Утечка данных

Утечкой данных называется ситуация, когда в процессе обучения модели используется информация, которая не будет доступна при ее последующем использовании. Это приводит к **завышению** оценки **качества** модели.

Самый простой пример утечки данных - это **дублирование** одних и тех же объектов в **train** и **test** выборках. 

 #### Дублирование данных 

Дублирование данных часто случается при сборе данных из различных источников. Посмотрим, чем оно опасно. 

Для **примера** возьмем 10 картинок из CIFAR10. Будем считать это train данными.

In [ ]:
import numpy as np

from torchvision import datasets
from sklearn.model_selection import train_test_split


np.random.seed(42)

dataset = datasets.CIFAR10("content", train=True, download=True)

data, _, labels, _ = train_test_split(dataset.data / 255,   # normalize
                                      np.array(dataset.targets),
                                      train_size=10,        # get only 10 imgs
                                      random_state=42,
                                      stratify=dataset.targets)
print(data.shape)

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(nrows=2, ncols=5, figsize=(10, 5))
for i in range(10):
    axs[i//5][i%5].imshow(data[i])
    axs[i//5][i%5].set_title(labels[i])
plt.show()

Предположим картика из train оказалась в test. Выберем картинку из этих 10 и применим алгоритм k-nearest neighbors с k=1.

In [ ]:
x_test = data[3]

# L1 distance
def compute_L1(a, b):
    return np.sum(np.abs(a - b))

# distance calculation
distances = []
for i in range(10):
    l1 = compute_L1(x_test, data[i])
    distances.append(l1)

distances = np.array(distances)
print(distances)

In [ ]:
indx = np.argmin(distances)
print(indx)

In [ ]:
data_test, _, labels_test, _ = train_test_split(dataset.data / 255,   # normalize
                                      np.array(dataset.targets),
                                      train_size=10,        # get only 10 imgs
                                      random_state=24,
                                      stratify=dataset.targets)

Ближайшим соседом для картинки, просочившейся в test стала эта же картинка.

Если все данные из test будут присутствовать в train, то мы просто будем искать эту же картинку в train с чем алгоритм k-nearest neighbors с $k=1$ справляется идеально. Итогом станет $accuracy = 1$ на выходе. Но с применением на незнакомой картинке результат будет хуже. 

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=5, figsize=(10, 5))
for i in range(10):
    axs[i//5][i%5].imshow(data_test[i])
    axs[i//5][i%5].set_title(labels_test[i])
plt.show()

In [ ]:
x_test = data_test[1]

# distance calculation
distances = []
for i in range(10):
    l1 = compute_L1(x_test, data[i])
    distances.append(l1)

distances = np.array(distances)
print(distances)

indx = np.argmin(distances)
print(labels[indx])

Смотрим картинки с train. Ближайшим соседом для кота стала лягушка. 

**Если вы получили $accuracy = 1$, то, скорее всего, вы что-то делаете не так!**

Дубли в данных следует удалить.

**Пример** поиска очевидных дубликатов в данных в датасете ["DOHMH Dog Bite Data"](https://data.cityofnewyork.us/Health/DOHMH-Dog-Bite-Data/rsgh-akpg)

In [ ]:
# Download dataset
!wget https://data.cityofnewyork.us/api/views/rsgh-akpg/rows.csv?accessType=DOWNLOAD -O dogs.csv

# Load into pandas and display a sample
import pandas as pd
dataset = pd.read_csv('dogs.csv')
dataset.head()

In [ ]:
if len(dataset) == len(dataset.drop_duplicates()):
    print('No duplicates')
else:
    print('%.2f percent of data is duplicate' 
          % len(dataset.drop_duplicates())/len(dataset) * 100)

#### Утечка, спрятанная в признаках 

Часто данные, используемые для обучения могут содержать “подсказки” для модели, которых не будет в реальных данных.

Самый простой **пример**: таблица со столбцом - порядковым номером строки `row_number`, в которую сначала записали все данные, принадлежащие отрицательному классу, а потом все данные, принадлежащие положительному. Если не удалить этот столбец из данных, то вместо выделения сложных закономерностей модель будет искать решение в виде `if row_number > N`. В реальных данных записи не будут упорядочены. 


**Примером** датасета в котором нумерация строк может “все испортить” является, например, [Iris](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris). Посмотрим на значения target этого набора данных. 

In [ ]:
import sklearn.datasets

iris = sklearn.datasets.load_iris()
print(iris.target)

Утечка часто может прятаться в метаданных записи: столбец id, название файла, время записи/загрузки.

Иногда “подсказки” спрятаны внутри признаков, поэтому важно понимать с какими данными вы работаете. 

**Пример:** у вас есть истории болезней пациентов с подозрением на онкологию. Необходимо решить задачу постановки диагнозов для новых пациентов. Вы обучаете модель и получаете подозрительно хороший результат. Начинаете разбираться и понимаете, что в данных, на которых обучалась модель присутствует отметка о назначении пациенту химиотерапии, которая назначается только после определения диагноза. 

### Shortcut learning 

Shortcut learning - это термин, которым обозначается ситуация, когда модель принимает правильное решение по неправильной причине: “right for the wrong reasons".
Очень важно, чтобы данные, на которых происходит обучение были **репрезентативными** и модель не пыталась сделать предсказание по косвеным признакам. 

**Пример**: рассмотрим датасет фотографий объектов, где изображения в обучающем и тестовом наборе были получены на открытом воздухе в солнечный день. Наличие одинаковых погодных условий означает, что тестовое множество не будет независимым, а поскольку оно не охватывает более широкий спектр погодных условий, оно также не будет репрезентативным.

**Пример:** необходимо было обучить нейросеть отличать хаски от волка. Для обучения были выбраны фотографии хаски на фоне зелени и волков на фоне снега, вместо того, чтобы учиться отличать мордочки нейросеть научилась делать предсказание по фону. 

<center><img src ="https://edunet.kea.su/repo/EduNet-content/EXTRA_publish/img_license/husky.png" width="850"></center>
<center><i>Первая строка - обучающие данные (обратите внимания, что все хаски были сфотографированы летом, а волки зимой). Вторая строка - тестовые данные (нейросеть выучила, что лето - это признак хасок, по этому она не правильно предсказывает класс волков).</center></i>

Аналогичные ситуации могут возникать, когда для сбора обучающих и тестовых данных используется разное оборудование. **Например**, для обучения вы используете томограф из одного госпиталя, а оценку качества модели проводите на томографе из другого госпиталя. Если модель не учитывает характеристики оборудования, она, скорее всего, не будет обобщаться на томограф из второй больницы (при этом на своем собственном датасете из первого госпиталя, эту ошибку вы отловить не сможете).

### Разделение на train и test

#### Перемешивание данных


Как мы уже упоминали метки классов в датасете могут быть распределены не равномерно. Для того, чтобы сохранить соотношение классов при разделении на train и test необходимо указать параметр `stratify` при разбиении.

Еще одним параметром, используемым при разбиении, является `shuffle` (значение по умолчанию `True`). При `shuffle = True` датасет перед разбиением перемешивается.

Для наглядности будем делить датасет пополам. 

In [ ]:
def count_lables(lables):
    lable_count = {}
    for item in lables:
        if item not in lable_count:
            lable_count[item] = 0
        lable_count[item] += 1
    return lable_count

def print_split_stat(X_train, X_test, y_train, y_test):
    print("Train labels: ", y_train)
    print("Test labels:  ", y_test)
    print("Train statistics: ", count_lables(y_train))
    print("Test statistics:  ", count_lables(y_test))

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

data, labels = load_iris(return_X_y=True)
print("DataSet labels: ", labels)
print("DataSet statistics: ", count_lables(labels))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, train_size=0.5,
                                                    shuffle = False, 
                                                    random_state=42)

print_split_stat(X_train, X_test, y_train, y_test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, train_size=0.5, 
                                                    random_state=42)

print_split_stat(X_train, X_test, y_train, y_test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, train_size=0.5, 
                                                    random_state=42, stratify=labels)

print_split_stat(X_train, X_test, y_train, y_test)

В некоторых случаях данные нельзя перемешивать. Это касается задач в которых мы пытаемся предсказать будущее. В таких задачах train должен предшествовать test по времени. Более подробно об этом будет рассказано в лекции про рекуррентные нейронные сети. 


#### Данные из различных источников

При использовании данных из различных источников нужно учитывать это при разбиении. 

**Пример:** вы анализируете данные ЭКГ на предмет патологий. У вас есть три источника данных:
* аппарат ЭКГ в кардиологическом отделении (много патологий),
* аппарат ЭКГ, который используют на медосмотрах (мало патологий),
* аппарат ЭКГ из приемного покоя больницы (среднее число патологий). 

Каждый прибор имеет свои особенности: характерные шумы, точность измерения и т.п. Если модель научится определять с какого прибора пришли данные она получит “подсказку”. Хорошим решением будет оставить данные с аппарата ЭКГ из приемного покоя больницы для test, а обучаться только на данных с аппарата ЭКГ в кардиологическом отделении и аппарата ЭКГ, который используют на медосмотрах. Это позволит оценить, как обученная модель работает с “незнакомым" прибором.


### Дисбаланс классов 

todo

## Проблема подбора гиперпараметров на тестовой выборке
(Разбираем ДЗ, выясняем что:

а) у KNN есть гиперпараметры (расстояние, K)

б) Точность(+/- 3%) сильно зависит от разбиения
Первый пункт позволяет ввести понятие гиперпараметра,второй — кросс валидации.)

### Задание 4. Nearest Neighbors для картинок

Загрузим датасет с помощью функций torchvision, пока можно не очень вдумываться как это работает, дальше в курсе мы познакомимся с PyTorch значительно ближе

In [ ]:
from torchvision import datasets
import numpy as np

from sklearn.model_selection import train_test_split

dataset = datasets.CIFAR10("content", train=True, download=True)

np.random.seed(42)
data, _, labels, _ = train_test_split(dataset.data / 255,  # Normalize
                                      np.array(dataset.targets),
                                      train_size=0.1, # get only fraction of the dataset
                                      random_state=42,
                                      stratify=dataset.targets)


In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(nrows=2, ncols=5, figsize=(10, 5))
for i in range(10):
    axs[i//5][i%5].imshow(data[i])
    axs[i//5][i%5].set_title(labels[i])
plt.show()

Посмотрим что это за датасет такой

In [ ]:
data[0]

In [ ]:
data.shape

CIFAR - 4х мерный массив (N, W, H, C). $N$ - количество картинок, $W$ - ширина картинки, $H$ - высота картинки, $C$ - количество каналов (RGB).

Создайте subplots с 2мя строками и 2мя колонками и отобразите 4 любых картинки из `data`. 
Используйте `plt.imshow()`

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))

ax[0, 0].imshow(data[0])
ax[0, 1].imshow(data[1])
ax[1, 0].imshow(data[2])
ax[1, 1].imshow(data[3])
plt.show()

Разбейте датасет на тренировочный и тестовый наборы. Укажите аргументы `random_state=42`, `stratify=labels`

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, labels,
                                                    random_state=42,
                                                    stratify=labels)

print("X_train", X_train.shape)
print("X_test", X_test.shape)

Возьмите первую картинку из тестового набора и найдите ее ближайшего соседа из тренировочного

In [ ]:
def compute_L1(a, b):
    return np.sum(np.abs(a - b))

In [ ]:
a = X_test[0]

l1s = []
for i in range(len(X_train)):
    l1 = compute_L1(a, X_train[i])
    l1s.append(l1)

distances = np.array(l1s)

In [ ]:
indx = np.argmin(distances)
print(indx)

**Отобразите эти картинки на subplots с ncols=2**

In [ ]:
fig, ax = plt.subplots(ncols=2)
ax[0].imshow(X_test[0])
ax[1].imshow(X_train[indx])
plt.show()

**Посмотрите какой класс предсказывается**

In [ ]:
class_pred = y_train[indx]
class_to_idx = dataset.class_to_idx

print(list(class_to_idx.keys())[list(class_to_idx.values()).index(class_pred)])

Возьмите первую картинку из тестового набора и найдите K ее ближайших соседей (KNN) из тренировочного

In [ ]:
k = 5
indx = np.argsort(distances)[:k]

Отобразите ближайших соседей в виде subplots

In [ ]:
fig, ax = plt.subplots(ncols=k, figsize=(20, 5))
ax[0].imshow(X_test[0])
for i in range(1, k):
    ax[i].imshow(X_train[indx[i]])
plt.show()

Посчитайте KNN для всего датасета

Чем больше данных - тем дольше процесс. Реализуйте функцию для расчета расстояний. Если вы используете `for loops` - сделайте к ним *progress bars* с помощью [tqdm](https://github.com/tqdm/tqdm)

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
def compute_distances(train, test, distance_func):
    train_size = len(train)
    test_size = len(test)
    distances = np.full((test_size, train_size), np.inf)
    for i in tqdm(range(test_size)):
        for j in range(train_size):
            distances[i, j] = distance_func(test[i], train[j])

    return distances

In [ ]:
distances = compute_distances(X_train, X_test, compute_L1)

Теперь найдите k ближайших соседей и предскажите класс. [scipy.stats.mode](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mode.html)

In [ ]:
from scipy.stats import mode


def get_accuracy(distances, train_labels, test_labels, k):
    indexes = np.argsort(distances, axis=1)[:, :k]
    labels_of_top_classes = train_labels[indexes]
    predicted_class, _ = mode(labels_of_top_classes, axis=1)
    accuracy = np.mean(test_labels == predicted_class.flatten())
    return accuracy

In [ ]:
accuracy = get_accuracy(distances, y_train, y_test, k)
print(f'Accuracy = {accuracy * 100:.0f}%')

**Посчитайте точность для k=1..100 и постройте график точности от k**

In [ ]:
acc = []
for k in range(1, 100):
    acc.append(get_accuracy(distances, y_train, y_test, k))

In [ ]:
plt.plot(np.arange(1, 100), acc)
plt.show()

Поменяйте расстоянние L1 на L2 и сравните точность на всем датасете.

In [ ]:
def compute_L2(a, b):
    return np.sqrt(np.sum((a - b)**2))

In [ ]:
distances_l2 = compute_distances(X_train, X_test, compute_L2)

In [ ]:
acc_l2 = []
for k in range(1, 100):
    acc_l2.append(get_accuracy(distances_l2, y_train, y_test, k=k))

In [ ]:
plt.plot(np.arange(1, 100), acc, label='L1')
plt.plot(np.arange(1, 100), acc_l2, label='L2')
plt.legend()
plt.show()

### Оптимизируйте гиперпараметры вашей модели

Многие модели имеют **гиперпараметры** - то есть числа или параметры, которые влияют на конфигурацию модели. Примерами могут служить: функция ядра, используемая в SVM; количество деревьев в случайном лесу и архитектура нейронной сети. Многие из этих гиперпараметров существенно влияют на производительность модели, и, как правило, универсальных гиперпараметров (таких, которые были бы оптимальны для всех возможных задач) не существует.

То есть, чтобы получить максимальную отдачу от модели, гииперпараметры нужно подбирать под конкретный набор данных. Хотя может возникнуть соблазн возиться с гиперпараметрами до тех пор, пока вы не найдете что-то подходящее, такой подход, скорее всего, не будет оптимальным. Гораздо лучше использовать какую-то стратегию **оптимизации гиперпараметров** (в качестве бонуса, обоснованная стратегия, в публикации смотрится значительно лучше, чем что-то в стиле *hyperparameters were chosen by chance*). Базовые стратегии включают случайный поиск и поиск по сетке, но они не очень хорошо масштабируются для большого количества гиперпараметров или для моделей, которые дорого обучать, поэтому стоит использовать инструменты, которые ищут оптимальные конфигурации более интеллектуальным способом (исследовано в [Yang et al., 2020](https://arxiv.org/abs/2007.15745)).

#### Библиотеки для оптимизации гиперпараметров

Существует довольно много библиотек для оптимизации гиперпараметров. Ключевыми являются [Ray-tune](https://docs.ray.io/en/latest/tune/index.html), [Optuna](https://optuna.readthedocs.io/en/stable/) и [Hyperopt](https://github.com/hyperopt/hyperopt). В целом они друг от друга принципиально не отличаются, так что скорее это вопрос вкуса. В качестве примера (и для разнообразия) рассмотрим библиотеку `Ray-Tune`.

##### **Ray-Tune**

In [ ]:
!pip install ray[tune] tune-sklearn

In [ ]:
from ray.tune.sklearn import TuneGridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.datasets import load_iris
import numpy as np

iris = load_iris()
X = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 14)

# Example parameters to tune from SGDClassifier
parameter_grid = {"alpha": [1e-4, 1e-1, 1], "epsilon": [0.01, 0.1]}

tune_search = TuneGridSearchCV(
    SGDClassifier(),
    parameter_grid,
    early_stopping=True,
    max_iters=10)

tune_search.fit(x_train, y_train)

#best set of perameter
print(tune_search.best_params_)

#best score with best set of perameters
print(tune_search.best_score)

Запомните, когда вы оптимизируете гиперпараметры или признаки, используемые моделью, для их отбора нужно использовать отдельный, валидационный датасет, или же кросс-валидацию. Вы **НЕ должны подбирать модель или ее гиперпараметры на тесте**.

Также можно использовать методы **AutoML** для оптимизации выбора модели и ее гиперпараметров (см. обзор в [He et al., 2021](https://arxiv.org/abs/1908.00709)).

### Нормализация данных

Загрузим датасет с образцами здоровой и раковой ткани. Датасет состоит из 569 примеров, где каждой строчке из 30 признаков, соответствует класс `1` злокачественной (*malignant*) или `0` доброкачественной (*benign*) ткани. Задача состоит в том, чтобы по 30 признакам обучить модель определять тип ткани (злокачественная или доброкачественная).

Можно иметь сколь угодно хороший алгоритм для классификации - но до тех пор, пока данные на входе - мусор, на выходе из нашего чудесного классификатора мы тоже будем получать мусор (*garbage in - garbage out*). Давайте разберемся, что конкретно надо сделать, чтобы kNN реально заработал.


In [ ]:
import sklearn.datasets

cancer = sklearn.datasets.load_breast_cancer() # load data
X = cancer.data # features
Y = cancer.target # labels(classes)
print(f'X shape: {X.shape}, Y shape: {Y.shape}') 
print(f'X[0]: \n {X[0]}') 
print(f'Y[0]: \n {Y[0]}') 

Посмотрим сколько данных в классе `0` и сколько данных в классе `1`

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5)) # set fig size 
plt.bar(1,Y[Y==1].shape, label=cancer.target_names[0]) # 1 label 
plt.bar(0,Y[Y==0].shape, label=cancer.target_names[1]) # 0 label
plt.title('Class balance') 
plt.ylabel('Num examples') 
plt.xticks(ticks=[1,0], labels=['1','0']) 
plt.legend(loc='upper left') 
plt.show() 

Теперь давайте посмотрим на сами данные. У нас есть 569 строк в каждой, из которой, по 30 колонок. Такие колонки называют признаками или *features*. Попробуем математически описать все эти признаки (mean, std, min и тд)

In [ ]:
import pandas as pd
pd.DataFrame(X).describe()

То же самое, но в виде графика. Видно, что у фич совершенно разные значения.

In [ ]:
import seaborn as sns

ax = sns.boxenplot(data=pd.DataFrame(X), orient="h", palette="Set2")
ax.set(xscale='log', xlim=(1e-4, 1e4), xlabel='Values', ylabel='Features')
plt.show()

Чтобы адекватно сравнить данные между собой нам следует использовать нормализацию.

 **Нормализация, выбор Scaler**

Нормализация — это преобразование данных к неким безразмерным единицам.
Ключевая цель нормализации — приведение различных данных в самых разных единицах измерения и диапазонах значений к единому виду, который позволит сравнивать их между собой.

Главное условие правильной нормализации — все признаки должны быть равны в возможностях своего влияния.

Например, у нас есть данные по группе людей: *возраст* (в годах) и *размер дохода* (в рублях). Возраст может измениться в диапазоне от 18 до 70 ( интервал 70-18 = 52). А доход от 30 000 р до 500 000 р (интервал 500 000 - 30 000 = 470 000). В таком варианте разница в возрасте имеет меньшее влияние, чем разница в доходе. Получается, что доход становится более важным признаком, изменения в котором влияют больше при сравнении схожести двух людей.

Должно быть так, чтобы максимальные изменения любого признака в «основной массе объектов» были одинаковы. Тогда потенциально все признаки будут равноценны.

Осталось определиться с выбором инструмента, часто используют следующие варианты: `MinMaxScaler`, `StandardScaler`, `RobustScaler`

Сравним `MinMaxScaler`, `StandardScaler`, `RobustScaler` для признака `data[:,0]`. **Обратите внимание на ось X**

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import numpy as np

np.random.seed(42)  # setting the initialization parameter for random values

# generate random values from 1 to 255, shape (30,1)
test = X[:,0].reshape(-1,1)

plt.figure(1, figsize=(30, 5))  
plt.subplot(141)  # set location
plt.scatter(test, range(len(test)), c=Y)  
plt.ylabel("Num examples", fontsize=15)  
plt.xticks(fontsize=15)  
plt.yticks(fontsize=15)  
plt.title("Non scaled data", fontsize=18)  

# scale data with MinMaxScaler
test_scaled = MinMaxScaler().fit_transform(test)  
plt.subplot(142)
plt.scatter(test_scaled, range(len(test)), c=Y)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("MinMaxScaler", fontsize=18)

# scale data  with StandardScaler
test_scaled = StandardScaler().fit_transform(test)  
plt.subplot(143)
plt.scatter(test_scaled, range(len(test)), c=Y)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("StandardScaler", fontsize=18)

# scale data  with RobustScaler
test_scaled = RobustScaler().fit_transform(test)  
plt.subplot(144)
plt.scatter(test_scaled, range(len(test)), c=Y)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("RobustScaler", fontsize=18)
plt.show()

Идея **`MinMaxScaler`** заключается в том, что он преобразует данные в диапазоне от 0 до 1. Может быть полезно, если нужно выполнить преобразование, в котором отрицательные значения не допускаются (e.g., масштабирование RGB пикселей)


$$z=\frac{X_i-X_{min}}{X_{max}-X_{min}}$$

$X_{min}$ и $X_{max}$ задаются как минимальное и максимальное допустимое значение, по умолчанию:  $X_{min}=0$  и $X_{max}=1$

Идея **`StandardScaler`** заключается в том, что он преобразует данные таким образом, что распределение будет иметь среднее значение 0 и стандартное отклонение 1. Большинство значений будет в  диапазоне от -1 до 1. Это стандартная трансформация, и она применима во многих ситуациях.

$$z=\frac{X-u}{s}$$

$u$ — среднее значение (или 0 при `with_mean=False`) и $s$ — стандартное отклонение (или 0 при `with_std=False`)

И StandardScaler и MinMaxScaler очень чувствительны к наличию выбросов. **`RobustScaler`** использует медиану и основан на *процентилях*. Процентиль — мера, в которой процентное значение общих значений равно этой мере или меньше ее. Например, 90 % значений данных находятся ниже 90-го процентиля, а 10 % значений данных находятся ниже 10-го процентиля. Соответственно, RobustScaler не зависит от небольшого числа очень больших предельных выбросов (outliers). Следовательно, результирующий диапазон преобразованных значений признаков больше, чем для предыдущих скэйлеров и, что более важно, примерно одинаков.

$$z=\frac{X-X_{median}}{IQR}$$

$X_{median}$ — значение медианы, $IQR$ — межквартильный диапазон равный разнице между 75-ым и 25-ым процентилями

Для нашей задачи по определению раковых опухолей обработаем наши 30 признаков с помощью StandardScaler.

In [ ]:
X_norm = StandardScaler().fit_transform(X)  # scaled data

Видим что они стали намного более сравнимы между собой.

In [ ]:
pd.DataFrame(X_norm).describe()

In [ ]:
ax = sns.boxenplot(data=pd.DataFrame(X_norm), 
                   orient="h", 
                   palette="Set2")
ax.set(xlabel='Values', ylabel='Features')
plt.show()

### Переобучение

Теперь обучим kNN для общей выборки данных, при разном значении количества соседей.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

n_nei_rng = np.arange(1, 31)  # array of the number of neighbors

quality = np.zeros(
    n_nei_rng.shape[0]
)  

for ind in range(n_nei_rng.shape[0]):  # for all elements
    # create knn for all num neighbors 
    knn = KNeighborsClassifier(
        n_neighbors=n_nei_rng[ind]
    )  
    knn.fit(X_norm, Y)  
    q = accuracy_score(y_pred=knn.predict(X_norm), y_true=Y)  # accuracy
    quality[ind] = q  # fill quality

plt.figure(figsize=(8, 5))  
plt.title("KNN on train", size=20)  
plt.xlabel("Neighbors", size=15)  
plt.ylabel("Accuracy", size=15)  
plt.plot(n_nei_rng, quality)  
plt.xticks(n_nei_rng) 
plt.show()  

Видим, что качество на 1 соседе - самое лучшее. Но это и понятно - ближайшим соседом элемента из обучающей выборки будет сам объект. Мы просто **запомнили** все объекты.

Если теперь мы попробуем взять какой-то новый образец опухоли и классифицировать его - у нас скорее всего ничего не получится. В таких случаях мы говорим, что наша модель не умеет обобщать (*generalization*).

Для того, чтобы знать заранее обобщает ли наша модель или нет, мы можем разбить все имеющиеся у нас данныe на 2 части. Но одной части мы будем обучать классификатор (*train set*), а на другой тестировать насколько хорошо он работает (*test set*).

In [ ]:
from sklearn.model_selection import train_test_split

# split data to train/test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)

scaler = StandardScaler()  
scaler.fit(X_train)  
X_train_norm = scaler.transform(X_train)  # scaling data
X_test_norm = scaler.transform(X_test)  # scaling data

n_nei_rng = np.arange(1, 31)  
train_quality = np.zeros(n_nei_rng.shape[0])  # quality on train data
test_quality = np.zeros(n_nei_rng.shape[0])  # quality on test data

for ind in range(n_nei_rng.shape[0]):  
    knn = KNeighborsClassifier(n_neighbors=n_nei_rng[ind])  
    knn.fit(X_train_norm, Y_train)  
    
    # accuracy on train data
    trq = accuracy_score(y_pred=knn.predict(X_train_norm), y_true=Y_train)  
    train_quality[ind] = trq  

    # accuracy on test data
    teq = accuracy_score(y_pred=knn.predict(X_test_norm), y_true=Y_test)  
    test_quality[ind] = teq  

# accuracy plot  on train and test data
plt.figure(figsize=(8, 5))
plt.title("KNN on train vs test", size=20)
plt.plot(n_nei_rng, train_quality, label="train")
plt.plot(n_nei_rng, test_quality, label="test")
plt.legend()
plt.xticks(n_nei_rng)
plt.xlabel("Neighbors", size=15)
plt.ylabel("Accuracy", size=15)
plt.show()

Вот, теперь мы видим, что 1 сосед был "ложной тревогой". Такие случаи мы называем *переобучением*. Чтобы действительно предсказывать что-то полезное, нам надо выбирать число соседей, начиная минимум с 3

# Кросс-валидация

## Алгоритм кросс валидации

Каждая модель имеет как ряд **параметров**, которые она меняет в процессе обучения (например, веса модели), так и ряд **гиперпараметров**, которые влияют на то, каким способом модель меняет параметры в процессе обучения. 

В случае kNN параметры, строго говоря, отсутствует - модель просто запоминает объекты обучающей выборки. Особо упорные могут считать их параметрами. 

А вот гиперпараметры есть, даже несколько групп. Какие? 

1. Число соседей 
2. Функция, которой считаем расстояние между объектами (L2=eucledian, L1=manhattan)
3. Веса, с которыми складываем метки ближайших соседей
4. Признаки! (но об этом с вами поговорим позже)
5. Сама модель - мы могли выбрать не kNN, а нагуглить что-нибудь другое

Посмотрим еще раз на график, который нарисовали на прошлом шаге. Какое число соседей считать оптимальным? Метрика явно скачет? 

In [ ]:
plt.figure(figsize=(8, 5))
plt.title("KNN on train vs test", size=20)
plt.plot(n_nei_rng, train_quality, label="train")
plt.plot(n_nei_rng, test_quality, label="test")
plt.xlabel("Neighbors", size=15)
plt.ylabel("Accuracy", size=15)
plt.legend()
plt.xticks(n_nei_rng)
plt.show()


Не понятно, насколько результат зависит от того, как нам повезло или не повезло с разбиением данных на обучение и тест. Может оказаться так, что для конкретного разбиения хорошо выбрать k=5, а для другого - k=7. 

Кроме того, опять же - фактически, мы сами выступаем в роли модели, которая учит гиперпараметры (а не параметры) под видимую ей выборку. 

Представим себе, что у нас есть 10000 моделей, полученных подкручиванием разных гиперпараметров (в том числе, выбором просто разного типа модели). Представим, что все эти модели не работают. Вообще. Представим так же, что каждая модель угадывает класс в задаче разделения на два класса с вероятностью 0.5 (и будем считать, что классы у нас сбалансированны - то есть 50% одного класса и 50% другого). 
Опять же, понятно, что классификация такими моделями ничем не лучше подбрасывания монетки. 



In [ ]:
def guess_model(Y_real):
    # array with values True/False, for random choice by mask 
    guessed = np.random.choice(
        [True,False],  
        size=Y_real.shape[0],
        replace=True,
    )
    Y_pred = np.zeros_like(Y_real)  # zeros array, shape Y_real
    
    # with mask 'guessed' assign values 
    Y_pred[guessed] = Y_real[guessed]  
    Y_pred[~guessed] = 1 - Y_real[~guessed]  
    return Y_pred

In [ ]:
models_num = 10000  # num of experiments
best_quality = 0.5  # quality threshold for accuracy

# array with random values in range 0, 1
Y_real = np.random.choice(
    [0, 1], size=250, replace=True
)  

for i in range(models_num):  # for all expirements
    Y_pred = guess_model(Y_real)  # predicted values
    q = accuracy_score(y_pred=Y_pred, y_true=Y_real)  # accuracy
    if q > best_quality:  
        best_quality = q   
print(f"Best result {best_quality}")

То есть мы перебором всех возможных моделей вполне можем получить для абсолютно бесполезной модели приемлемое качество


Получается, что если подбирать гиперпараметры модели на *train set*, то:
1. Можно переобучитьcя, просто на более "высоком" уровне. Особенно если гиперпараметров у модели много и все они разнообразны
2. Нельзя быть уверенным, что выбор параметров не зависит от разбиения на обучение и тест 

Поэтому мы:

1. Подбираем гиперпараметры моделей на отдельном датасете, называемым валидационным. Получаем мы его разбиением обучающего датасета на собственно обучающий и валидационный 

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/split_dataset_for_train_val_test.png" width="700">

2. Чаще всего делаем несколько таких разбиений по какой-то схеме, чтобы получить уверенность оценок качества для моделей с разными гиперпараметрами - **кросс-валидация**

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/cross_validation_on_train_data.png" width="500">

Часто применяется следующий подход, называемый [K-Fold кросс-валидацией](https://scikit-learn.org/stable/modules/cross_validation.html):

Берется тренировочная часть датасета, разбивается на части - блоки. Дальше мы будем использовать для проверки первую часть (Fold 1), а на остальных учиться. И так последовательно для всех частей. В результате у нас будут информация о точности для разных фрагментов данных и уже на основании этого можно понять, насколько значение этого параметра, который мы проверяем, зависит или не зависит от данных. То есть если у нас от разбиения точность при одном и том же К меняться не будет, значит мы подобрали правильное К. Если она будет сильно меняться в зависимости от того, на каком куске данных мы проводим тестирование, значит, надо попробовать другое К и если ни при каком не получилось - то это такие данные.  


##Оценка результата кросс-валидации
(вообще не понятно, что это)

##Типичные ошибки при кросс-валидации

**Можно ли делать только кросс-валидацию (без теста)?**


Нет, нельзя. Кросс-валидация не до конца спасает от подгона параметров модели под выборку, на которой она проводится. Оценка конечного качества модели должно производиться на отложенной тестовой выборке. Если у вас очень мало данных, можно рассмотреть [вложенную кросс-валидацию](https://weina.me/nested-cross-validation/), Речь об этом пойдет позже, в последующих лекциях. Но даже в этом случае придется анализировать поведение модели, чтобы показать, что она учит что-то разумное. Кстати, вложенную кросс-валидацию можно использовать, чтобы просто получить более устойчивую оценку поведения модели на тесте.

## Кросс-валидация для научных исследований - на что обратить внимание

### Перекрестная валидация (*cross-validation*)

Если вы не можете получить больше данных - то вы можете более эффективно использовать уже имеющиеся, используя перекрестную валидацию (*cross validation*). 


Вообще говоря, кросс-валидация используется для более точной оценки модели. Одна оценка модели может быть ненадежной и может либо недооценить, либо переоценить истинный потенциал модели. По этой причине обычно проводится несколько оценок. Существует несколько способов провести множественную оценку модели, и большинство из них предполагает многократное обучение модели с использованием различных подмножеств обучающих данных. **Перекрсная валидация** (CV) особенно популярна и имеет множество разновидностей [Arlot et al., 2010](https://projecteuclid.org/journals/statistics-surveys/volume-4/issue-none/A-survey-of-cross-validation-procedures-for-model-selection/10.1214/09-SS054.full).

**Перекрсная валидация** в машинном обучении подразумеват, что вместо того, чтобы разделить наш набор данных на две части, одну для обучения (`train`), а другую для тестирования (`test`), мы разбиваем наш набор данных на несколько частей, обучаем на некоторых из них, а остальные используем для тестирования. Затем мы используем другие части для обучения и тестирования нашей модели. Это гарантирует, что наша модель обучается и тестируется на новых данных на каждом новом шаге.

Чаще всего используют десятикратную прекресную валидацию.

#### Разберем на примере

Создадим датасет

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=1000, noise=0.3, random_state=42)

plt.scatter(X[:,0], X[:,1], c=y)

И разделим его на 10 (*K*) складок (*folds*)

In [ ]:
classifier = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
X, y = make_moons(noise=0.3, random_state=42, n_samples=1000)

scores = []

# Lets test 100 times
for i in range(0,100):
    # For each experiment lets choose a different random state for splitting
    random_state = np.random.randint(0,2**32)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.4, random_state=random_state
    )

    # normalize data
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    # predict using each classifier
    classifier.fit(X_train, y_train)
    scores.append([random_state, classifier.score(X_test, y_test)])

In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

#Split data intto 10 folds
kf = KFold(n_splits=10)
kf.get_n_splits(X)

#Define scaler and classifier
scaler = MinMaxScaler()
clf = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)

#Define figure space
fig,ax = plt.subplots(nrows=2, ncols=5, figsize=(10,4))

row = 0
scores = []

#Itterate over folds
for col, (train_index, test_index) in enumerate(kf.split(X)):
    #Split
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    #Normalize
    scaler.fit(X_train)
    scaler.fit(X_test)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    #Classify
    clf.fit(X_train, y_train)

    #Gauge performance
    score = clf.score(X_test, y_test)
    scores.append(score)

    #Plot figure
    if col > 4:
        col-=5
        row=1

    ax[row, col].scatter(X_train[:,0], X_train[:,1], c=y_train, alpha=0.05)
    ax[row, col].scatter(X_test[:,0], X_test[:,1], c=y_test, marker='x')
    ax[row, col].set_title(score)
    ax[row, col].axis('off')

Результирующую точность модели мы определим как среднее от всех складок

In [ ]:
print('Финальная точность = %.2f ± %.2f' % (np.mean(scores), np.std(scores)))

Для б**о**льшей надежности можно использовать метод **вложенной перекрестной валидации** (также известный как двойная кросс-валидация, (см. [Cawley, 2010](https://www.jmlr.org/papers/volume11/cawley10a/cawley10a.pdf) и [Wainer et al., 2021](https://research-portal.uea.ac.uk/en/publications/nested-cross-validation-when-selecting-classifiers-is-overzealous)).

##GridSearch

Подберем параметры для модели с помощью **GridSearchCV**.

GridSearchCV – это инструмент для автоматического подбирания параметров для моделей машинного обучения. GridSearchCV находит наилучшие параметры, путем обычного перебора: он создает модель для каждой возможной комбинации параметров

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from IPython.display import clear_output
"""
Parameters for GridSearchCV:
estimator — model
cv — num of fold to cross-validation splitting 
param_grid — parameters names
scoring — metrics 
n_jobs - number of jobs to run in parallel, -1 means using all processors.
"""
model = GridSearchCV(
    estimator=KNeighborsClassifier(),
    cv=KFold(3, shuffle=True, random_state=42),
    param_grid={
        "n_neighbors": np.arange(1, 31),
        "metric": ["euclidean", "manhattan"],
        "weights": ["uniform", "distance"],
    },
    scoring="accuracy",
    n_jobs=-1,
)
model.fit(X_train_norm, Y_train)
clear_output()

Выведем лучшие гиперпараметры для модели, которые подобрали:

In [ ]:
print("Metric:", model.best_params_["metric"])
print("Num neighbors:", model.best_params_["n_neighbors"])
print("Weigths:", model.best_params_["weights"])


Объект GridSearchCV можно использовать как обычную модель

In [ ]:
from sklearn.metrics import balanced_accuracy_score

Y_pred = model.predict(X_test_norm)
print(f"Percent correct predictions {np.round(accuracy_score(y_pred=Y_pred, y_true=Y_test)*100,2)} %")
print(f"Percent correct predictions(balanced classes) {np.round(balanced_accuracy_score(y_pred=Y_pred, y_true=Y_test)*100,2)} %")

Мы можем извлечь дополнительные данные о кроссвалидации и по ключу обратиться к результатам всех моделей

In [ ]:
list(model.cv_results_.keys())

Выведем для примера mean_test_score:

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(model.cv_results_["mean_test_score"])
plt.title("mean_test_score", size=20)
plt.xlabel("Num of experiment", size=15)
plt.ylabel("Accuracy", size=15)
plt.show()

Построим, например, при фиксированных остальных параметрах (равных лучшим параметрам), качество модели на валидации в зависимости от числа соседей

In [ ]:
selected_means = []
selected_std = []
n_nei = []
for ind, params in enumerate(model.cv_results_["params"]):
    if (
        params["metric"] == model.best_params_["metric"]
        and params["weights"] == model.best_params_["weights"]
    ):
        n_nei.append(params["n_neighbors"])
        selected_means.append(model.cv_results_["mean_test_score"][ind])
        selected_std.append(model.cv_results_["std_test_score"][ind])

Построим error bar, для сравнения разброса ошибки при разном количестве соседей Neighbors

Видим, что на самом деле большой разницы в числе соседей и нет. 

In [ ]:
plt.figure(figsize=(8, 5))
plt.title(f"KNN CV, {params['metric']}, {params['weights']}", size=20)
plt.errorbar(n_nei, selected_means, yerr=selected_std, linestyle="None", fmt="-o")
plt.xticks(n_nei)
plt.ylabel("Mean_test_score", size=15)
plt.xlabel("Neighbors", size=15)

plt.show()

##RandomGridSearch
(нету)

# Признаки


## Данные, требующие предобработки

### Потратьте время на понимание своих данных

В конечном итоге вы захотите опубликовать свою работу. Публиковать работы основанные на данных из надежного источника (собранных с использованием надежной методологии) значительно проще.

Если вы используете данные, скачанные с интернет-ресурса, **убедитесь, что вы знаете, откуда они взяты** (описаны ли они в статье? Если да, посмотрите на документ; убедитесь, что он был опубликован в авторитетном месте, и проверьте, упоминают ли авторы какие-либо ограничения данных).

Не предполагайте, что если набор данных использовался в ряде работ, то он хорошего качества - иногда данные используются только потому, что их легко достать, а некоторые широко используемые наборы данных, как известно, имеют существенные ограничения (см. [Paullada et al., 2020]((https://arxiv.org/abs/2012.05345))). **Например**, при исследовании категории `faces` в ImageNet ([Deng et al., 2009](https://ieeexplore.ieee.org/document/5206848), [Crawford & Paglen, 2019](https://excavating.ai/) ) обнаружили миллионы изображений людей, которые были помечены оскорбительными категориями, включая расистские и унизительные фразы. В ответ на эту работу, большая часть набора данных ImageNet была удалена ([Yang et al., 2020](https://dl.acm.org/doi/abs/10.1145/3351095.3375709)).

Если вы обучаете свою модель на плохих данных, то, скорее всего, у вас получится плохая модель: процесс, известный как **garbage in garbage out** (чушь на входе - чушь на выходе). Поэтому всегда начинайте с проверки, что ваши **данные имеют смысл**. 

Проведите **эксплораторный анализ данных** (см. [Cox, 2017](https://www.oreilly.com/library/view/translating-statistics-to/9781484222560/A426308_1_En_3_Chapter.html)). Ищите недостающие или непоследовательные записи. Гораздо проще сделать это сейчас, до обучения модели, чем потом, когда вы будете пытаться объяснить рецензентам, почему вы использовали плохие данные. 

Анализ важно провести независимо от того, используете ли вы существующие наборы данных или генерируете новые данные в рамках своего исследования (в этом случае учитывайте, что исследовательский анализ может быть ценен сам по себе, а результаты этого анализа могут стать важной частью вашей статьи).


Отдельным пунктом, хочется отметить, что помимо "содержания", важна и "форма" данных. **Формат хранения** ваших данных повлияет на **скорость, с которой вы сможете завершить свое исследование**. **Например**, у вас есть массив, который называется `ID` и в нем хранятся следующие данные `[1,30,111,221,234]` в формате `float64`. Проверьте, а точно ли тут нужен `float64`, возможно, ваши данные представлены целыми положительными числами, и для их хранения будет достаточно формата `uint32` или даже `uint16` (см. подробный обзор форматов данных в [Understanding Data Types](https://jakevdp.github.io/PythonDataScienceHandbook/02.01-understanding-data-types.html)).

#### Разберем на конкретном примере

Скачаем датасет: **"Когда и где кого-то покусала собака в NYC"** и загрузим его в `pandas`

In [ ]:
# Download dataset
!wget https://data.cityofnewyork.us/api/views/rsgh-akpg/rows.csv?accessType=DOWNLOAD -O dogs.csv

# Load into pandas and display a sample
import pandas as pd
dataset = pd.read_csv('dogs.csv')
dataset.head(5)

Проверим есть ли **дупликаты**:

In [ ]:
if len(dataset) == len(dataset.drop_duplicates()):
    print('Очевидных дупликатов нет')
else:
    print('%.2f процентов данных являются дупликатами' % len(dataset.drop_duplicates())/len(dataset) * 100)

У нас есть колонки: `UniqueID`, `DateOfBite`, `Species`,	`Breed`, `Age`, `Gender`, `SpayNeuter`, `Borough`, `ZipCode`, давайте проверим все ли с ними в порядке. Начнем с того, что определим в каком виде хранятся наши данные в памяти

In [ ]:
dataset.dtypes

`Object` - это далеко не самая эффективная форма хранения информации в `Pandas-DataFrame` и, как правило, отличный индикатор того, что с данными, что-то не так. Давайте разберемся с каждой колонкой по отдельности.

#### `UniqueID`

Мы ожидаем, что в этой колонке каждому объявлению был присвоен уникальный ID. Судя по сэмплу, это просто порядковый номер начинающийся с 1. Можем визуализировать эту колонку, что бы убедиться что там никаких сюрпризов.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.arange(len(dataset))
plt.scatter(x, dataset['UniqueID'], s=0.1)
plt.xlabel('Index')
plt.ylabel('UniqueID')

Можно заметить, что уникальных идентификаторов меньше чем строк в датафрейме. Давайте убедимся:

In [ ]:
dataset['UniqueID'].max(), len(dataset['UniqueID'])

То есть ID повторяются?

In [ ]:
dataset.sort_values('UniqueID').head(6)

Судя по всему, в какой-то момент времени нумерация была запущена заново. А значит ID совсем даже не unique => использовать эту колонку как уникальный идентификатор мы не можем. 

В каком формате хранятся данные в этой колонке?

In [ ]:
dataset['UniqueID'].dtype

В `int64` можно записывать целые числа в диапазоне от `-9223372036854775808` до `9223372036854775807`. Мы уже по графику видим, что знак нам не нужен, и что наше максимальное значение явно меньше. Определим какой у нас максимум.

In [ ]:
dataset['UniqueID'].min(), dataset['UniqueID'].max()

Значит нам подойдет `uint16` целое число без знака в диапазоне от  `0` до `65535`

In [ ]:
dataset_filtered = dataset.copy()
dataset_filtered['UniqueID'] = dataset['UniqueID'].astype('uint16')

Узнаем сколько памяти мы выиграли

In [ ]:
def resources_gain(column = 'UniqueID', orig_dataset=dataset, filtered_dataset=dataset_filtered):
    original_memory = orig_dataset[column].memory_usage(deep=True)
    memory_after_conversion = filtered_dataset[column].memory_usage(deep=True)

    return original_memory/memory_after_conversion

resources_gain(column='UniqueID', orig_dataset=dataset, filtered_dataset=dataset_filtered)

Теперь колонка `UniqueID` занимает в 4 раза меньше места (а значит и обрабатывается быстрее)

#### `DateOfBite`

В `DateOfBite` судя по всему записано время укуса, но в формате `str`. Нам было бы удобнее работать с `timestamps`

In [ ]:
dataset_filtered['DateOfBite'] = pd.to_datetime(dataset['DateOfBite'])

Оценим выигрыш в ресурсах

In [ ]:
resources_gain(column='DateOfBite', orig_dataset=dataset, filtered_dataset=dataset_filtered)

Теперь проверим нет ли каких-то странных дат:

In [ ]:
dataset_filtered['DateOfBite'].hist()

С датами все в порядке, кстати можно заметить, что во время Ковида собакам было явно меньше кого кусать =)

#### `Species`

Мы ожидаем, что в этом отчете сообщается только об укусах собак, убедимся

In [ ]:
dataset['Species'].unique()

Если у нас целая колонка в которой исключительно значение `DOG`, то зачем нам эта колонка? Правильно. Удаляем ее

In [ ]:
del dataset_filtered['Species']

#### `Breed`

Посмотрим на то, какие значения есть в этой колонке

In [ ]:
dataset['Breed'].unique()

Можно заметить, что тип нашего массива - `object`. Обычно так бывает, когда в массиве есть несколько разных типов данных (например `float` и `str`). Давайте найдем все значения, которые str не являются

In [ ]:
dataset['Breed'][dataset['Breed'].apply(lambda x: type(x) != str)].unique()

Ага - `NaN`. А выше мы уже видели что есть категория `UNKOWN`. Давайте поправим

In [ ]:
dataset_filtered['Breed'][dataset['Breed'].apply(lambda x: type(x) != str)] = 'UNKNOWN'

Теперь посмотрим на поправленный список

In [ ]:
np.sort(dataset_filtered['Breed'].unique()).tolist()[:15]

Сразу видно, что категории часто повторяются с опечатками. Править это придется в ручную (чем мы заниматься на этой лекции конечно не будем). Но для примера поправим опечатку в `ALASKAN MALMUTE` (заменим на `ALASKAN MALAMUTE`)

In [ ]:
dataset_filtered['Breed'][dataset['Breed'] == 'ALASKAN MALMUTE'] = 'ALASKAN MALAMUTE'

У нас есть ограниченное (хоть и большое) количество пород, с точки зрения памяти, их выгоднее хранить как категориальные признаки

In [ ]:
dataset_filtered['Breed'] = dataset_filtered['Breed'].astype('category')

Оценим выигрыш в производительности

In [ ]:
resources_gain(column='Breed', orig_dataset=dataset, filtered_dataset=dataset_filtered)

В рамках примера, будем считать что колонку почистили

#### `Age`

Ожидаем, что колонка будет в числовом формате, на деле видим `object`, давайте разбираться

In [ ]:
dataset['Age'].unique()[:15]

Ну тут только руками... Нам надо выбрать единую единицу измерения (например месяцы) и все привести к ней. Там где непонятно будем писать `NaN` (хотя лучше бы выяснить конечно)

In [ ]:
dataset_filtered['Age'][ (dataset['Age'] == '4') | 
                        (dataset['Age'] == '7') |
                        (dataset['Age'] == '6') |
                        (dataset['Age'] == '5') |
                        (dataset['Age'] == '8') |
                        (dataset['Age'] == '11') |
                        (dataset['Age'] == '3')] = np.nan #так как не понятно 4 чего
dataset_filtered['Age'][dataset['Age'] == '4Y'] = 4*12
dataset_filtered['Age'][dataset['Age'] == '5Y'] = 5*12
dataset_filtered['Age'][dataset['Age'] == '3Y'] = 3*12
#ну и так далее

#### `Gender`

Посмотрим какие есть варианты пола собаки

In [ ]:
dataset['Gender'].unique()

Ну хоть тут без сюрпризов, но для увеличения производительности тоже сконвертируем данные в категориальные признаки

In [ ]:
dataset_filtered['Gender'] = dataset['Gender'].astype('category')

In [ ]:
resources_gain(column='Gender', orig_dataset=dataset, filtered_dataset=dataset_filtered)

Кто чаще кусается?

In [ ]:
dataset_filtered['Gender'].hist()

#### `SpayNeuter`

`Spay/Neutter` - это была ли собака стерилизована. Мы ожидаем только True и False (хотя удивительно, что не колонки unknown)

In [ ]:
dataset['SpayNeuter'].unique()

А формат у нас и так уже `bool`, который занимает мало места. Значит с этой колонкой закончили

#### `Borough`

Boroughs - это что-то типа наших округов (например ЮЗАО)

In [ ]:
dataset['Borough'].unique()

Тоже переведем в категориальные признаки

In [ ]:
dataset_filtered['Borough'] = dataset['Borough'].astype('category')

In [ ]:
dataset_filtered['Borough'].hist()

In [ ]:
resources_gain(column='Borough', orig_dataset=dataset, filtered_dataset=dataset_filtered)

#### `ZipCode`

In [ ]:
dataset['ZipCode'].unique()[:15]

Видим, что есть категории `?` и `NaN`. Их можно объединить

In [ ]:
dataset_filtered['ZipCode'][dataset['ZipCode'] == np.nan] = '?'

Остальные значения тоже пусть будут категориальными (у многих значений есть 0 в начале, пусть  остается, что бы не вводить дополнительную путаницу)

In [ ]:
dataset_filtered['ZipCode'] = dataset['ZipCode'].astype('category')

In [ ]:
dataset_filtered.dtypes

Оценим насколько меньше места теперь занимает **весь** датасет

In [ ]:
dataset.memory_usage(deep=True).sum()/dataset_filtered.memory_usage(deep=True).sum()

На этом можем считать нашу чистку завершенной. 

## Предобработка данных и инструменты

## Типы признаков

### Типы признаков 

Традиционно признаки делятся на:




#### Вещественные 
Вещественные признаки бывают:

 * дискретные. Например - число лайков от пользователей
 
 <img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/discrete_features_social_media_likes.jpg" width="1000">


 * непрерывные. Например - температура


 <img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/continuous_features_thermometer.png" width="600">
 
Понятно, что разделение часто условное. Тот же возраст можно посчитать и дискретной переменной (пользователь всегда нам сообщает свои полные года), и непрерывной (возраст можно считать с любой точностью, но никто не будет) )


Также иногда вещественные признаки делят на относительные (считаются относительно чего-то, уже нормированные и тд)  и интервальные. 

#### Категориальные 



Значение -  принадлежность к какой-то из категорий. Традиционно делятся на сильно отличающиеся по свойствам:
 * упорядоченные (ординальные) - для каждой пары возможных категорий можем сказать, какая больше, а какая меньше. Например - класс места. Или размер одежды

<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/categorical_ordered_features.png" width="600">



 * неупорядоченные (номинальные) - категории между собой несравнимы. Обычно нельзя сказать, что красный телефон больше синего. Или что солнечная погода больше снежной

<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/categorical_unordered_features.png" width="900">



Часто мы сталкиваемся с бинарными категориальными признаками, для которых известно только две возможных категории (например, биологический пол человека). 


## Гнерация и преобразования признаков

### Преобразования признаков



#### Вещественных признаков 



#### Бинаризация 

Например, нам может быть неинтересно, сколько конкретно раз встретилось явление в наблюдении - главное, что оно вообще встретилось. Тогда мы просто превращаем наш вещественный признак в бинарный "было ли явление", и работаем уже с ним. 



#### Округление

Часто данные до нас доходят с очень высокой точностью после запятой. Нужно ли это нашей модели - часто нет. Иногда по факту два наблюдения не различаются  по этому признаку (разница в пределах статошибки), но по признаку их отличить можно. Это может приводить к переобучению. В таких случаях разумно признаки округлить. 

#### Bining (Бинирование)

Опять же, нам не интересны точные значения - например, что видео набрало 1000 лайков, а не 1001. 

К тому же, число просмотров/лайков некоторых видео может быть очень большим в сравнении с остальными, что будет приводить к неадекватному поведению. 
В итоге часть значений у нас встречается часто, а часть - очень редко. Это может приводить к неадекватному поведению модели. 




##### Fixed-width bining

Просто бьем наши значения по диапазонам фиксированной длины. Так часто поступают с возрастом. 


<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/fixed_width_binning.png" width="400">



##### Adaptive Binning

Это не всегда работает хорошо. Например, распределение зарплат у нас очень сильно скошено вправо. 


<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/adaptive_binning.png" width="400">



И бинирование с фиксированной длиной бина нам не поможет справиться с редкими значениями.

В этой ситуации помогает бинирование, например, по квантилям - когда границы бина представляют собой квантили. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/quantiles_binning.png" width="400">

#### Логарифмирование

С ситуацией, когда распределение скошено вправо работает и другой подход - прологаримфировать величину. 


<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/log_binning.png" width="400">


Обобщением этого подхода является [Box-Cox Transform](https://www.statisticshowto.com/box-cox-transformation/#:~:text=A%20Box%20Cox%20transformation%20is,a%20broader%20number%20of%20tests.), общей целью которой является придать данным вид более похожий на нормальное распределение, с которым работает бОльшее число моделей и сходимость лучше 


#### Категориальных признаков 

#### Label encoding 

Просто берем и каждой категории однозначно сопоставляем число. 
Очень простой способ, если признак ординальный - будет работать почти всегда. 

Если же наш признак - номинальный, то могут возникнуть проблемы. Мы не можем сказать, что салатовый больше красного (в большинстве случаев). Но модель ничего про это не знает и после нашего кодирования спокойно такие сравнения может производить. Это может приводить к более низкому качеству модели и выучиванию ею неправильной информации. Кроме того, например, деревьям решений, чтобы выделить в таком случае конкретную категорию придется делать сразу несколько действий, которые, в силу жадности алгоритма их построения, могут и не быть найдены



 Некоторые модели (например, lightgbm) автоматически могут перекодировать все правильно, если им сообщить, что переданный признак - категориальный. Для некоторых это придется делать вручную. 

#### One-hot encoding 

Схемой, которая часто используется на практике, является one-hot encoding. Он состоит том, что вместо одного категориального признака X создается набор бинарных категориальных признаков, которые отвечают на вопрос "X == C? ", где C пробегает все возможные значения категориального признака. 

Теперь чтобы обусловиться на конкретное значение категориального признака, дереву решений достаточно задать один вопрос. 


<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/one_hot_encoding.png" width="450">




Однако у такой схемы есть один минус - мы получаем линейно зависимые признаки. Это может плохо влиять на некоторые модели (для случая нейронных сетей - обычно нет, но полезно держать в голове). 


<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/problem_of_ohe.png" width="850">

Потому иногда одну из категорий исключают при кодировании, например, в примере выше можно исключить Fish, ведь если все три других признака-категории равны 0, то точно верно, что категория - Fish. 


#### Target encoding 

Кодируем каждую категорию каким-то численным параметром, характеризующим то, что мы предсказываем. Например, можно каждую категорию категориального признака заменять на среднее 

На самом деле, так просто делать нельзя, можно получить переобученную модель. Как делать - можете подробно посмотреть, к примеру, [здесь](https://github.com/Dyakonov/PZAD/blob/master/2020/PZAD2020_042featureengineering_07.pdf) или [здесь](https://www.coursera.org/lecture/competitive-data-science/concept-of-mean-encoding-b5Gxv?redirectTo=%2Flecture%2Fcompetitive-data-science%2Fconcept-of-mean-encoding-b5Gxv) 


#### Embedding

Можно научить вашу модель саму сопоставлять каждой категории некий вектор определенной размерности. Для этого вначале сопоставляем каждой категории случайный вектор заданной длины. А далее изменяем этот вектор как обычные веса. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/embedding.png" width="500">


#### Кодирование циклических категориальных признаков

В случае с такими признаками, как день недели или время суток, мы сталкиваемся с проблемой того, что нам нужно предложить кодирование, которое будет учитывать, что понедельник близок к воскресенью так же, как понедельник же ко вторнику, и тд. 

В случае деревьев решений и методов на них основанных можно "забить" - такие методы сами разберутся. Для некоторых других методов, тех же нейросетей, правильно кодирование может улучшить качество и сходимость. 



Давайте нанесем наши категории, например, дни недели - на окружность. Как это сделать? 
Пусть понедельнику соответствует 1, а воскресенью - 7. Далее посчитаем два таких вспомогательных признака по следующим формулам

In [ ]:
import numpy as np
weekdays = np.arange(1, 8) #create an array of weekdays
print(weekdays)
sina = np.sin(weekdays * np.pi * 2 / np.max(weekdays)) #feature 1
cosa = np.cos(weekdays * np.pi * 2 / np.max(weekdays)) #feature 2

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(7,7)) #Decide figure size
plt.scatter(sina, cosa) #Plot scatter of feature 1 vs feature 2
for  i, z in enumerate( ("Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun") ): #for each day in a week
  plt.text(sina[i], cosa[i], s=z) #add text labels to plot
  

Что делать дальше? А по сути, мы уже все сделали. Теперь расстояния между понедельником и вторником и воскресеньем и понедельником одинаковые:

In [ ]:
dist_mon_tue = (sina[1] - sina[0]) ** 2 + (cosa[1] - cosa[0]) ** 2 #distance between Monday and Tuesday
dist_sun_mon = (sina[6] - sina[0]) ** 2 + (cosa[6] - cosa[0]) ** 2 #distance between Sunday and Monday
print('Distance between Mon-Tue = %.2f' % dist_mon_tue)
print('Distance between Sun-Mon = %.2f' % dist_sun_mon)

то же самое верно и для любых отстоящих друг от друга на одинаковое число дней



In [ ]:
dist_mon_wed = (sina[2] - sina[0]) ** 2 + (cosa[2] - cosa[0]) ** 2 #distance between Monday and Wednesday
dist_fri_sun = (sina[4] - sina[6]) ** 2 + (cosa[4] - cosa[6]) ** 2 #distance between Friday and Sunday
print('Distance between Mon-Wed = %.2f' % dist_mon_wed)
print('Distance between Fri-Sun = %.2f' % dist_fri_sun)

Таким образом, циклические признаки можно кодировать парой признаков - sin и cos, полученных по схеме, описанной выше. 



Проблемы подхода

1. Деревья решений могут решить задачу и так. А такое кодирование им, наоборот, будет мешать, т.к. они работают с одним признаком за раз

2. Надо понимать, что важность исходной категориальной фичи неочевидным образом делится между двумя полученными из нее таким образом. 

3. В некоторых задачах one-hot работает лучше

#### Кодирование взаимодействия признаков

Признаки могут по-разному взаимодейстовать и некоторые модели в принципе не могут моделировать это взаимодействие. 


Взаимодействовать могут вещественные переменные и категориальные


<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/strength_vs_speed.png" width="550">

**Сила vs. скорость**

категориальные и категориальные 


<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/categorical_and_categorical.png" width="300">

вещественные и вещественные 


<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/real_and_real.png" width="300">


Могут быть и более высокоуровневые взаимодействия - взаимодействуют много разных признаков.

Взаимодействия могут быть самые разные - много способов кодировать. Например, добавлять в число признаков их произведение. 

#### Генерация признаков при помощи модели

Если у вас есть модель, обученная на другом датасете, можно генерировать признаки при помощи нее. Например, при помощи случайного леса


<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/generating_features_using_model.png" width="700">

**Генерация бинарного признакового пространства с помощью RandomForest**

Хорошие источники: 

1. [Feature Selection for High-Dimensional Data](https://www.springer.com/gp/book/9783319218571)
2. [How to Win a Data Science Competition: Learn from Top Kagglers](https://www.coursera.org/learn/competitive-data-science)
3. **Feature Engineering for Machine Learning: Principles and Techniques for Data Scientists Paperback** – April 14, 2018 by Alice Zheng , Amanda Casar
4. [Сайт](https://dyakonov.org/) и [курс](https://github.com/Dyakonov/PZAD) Дьяконова
5. Серия статей на towardsdatascience, [первая из серии](https://towardsdatascience.com/understanding-feature-engineering-part-1-continuous-numeric-data-da4e47099a7b)
6. [A Few Useful Things to Know About Machine Learning](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)
7. [Про кодирование циклических признаков](http://blog.davidkaleko.com/feature-engineering-cyclical-features.html)

 ## Визуализация признаков
 (то есть до этого делаем все без картинок!!!!)

## Нормализация данных
(ибо поврторение - мать заикания)

# Линейная регрессия

 ## Понятие линейной регрессии

#### Пример простой линейной регрессии

Теперь ненадолго отвлечемся от SVM и рассмотрим другую задачу. В этой задаче мы будем прогнозировать успеваемость студента, в зависимости от количества часов, которые он учил материал. Это простая задача линейной регрессии, поскольку она включает всего две переменные.

**Регрессия** - это статистический метод, используемый в финансах, инвестировании и других дисциплинах, который пытается определить силу и характер связи между одной зависимой переменной (обычно обозначаемой **Y**) и рядом других переменных (известных как независимые переменные). В задачах регрессии, мы будем пытаться минимизировать ошибку между предсказанием и истинными данными. 

"Регрессия" происходит от слова "регресс", которое, в свою очередь, происходит от латинского "regressus" - возвращаться (к чему-либо). В этом смысле регрессия - это техника, которая позволяет "вернуться назад" от беспорядочных, трудно интерпретируемых данных к более четкой и осмысленной модели.

Загрузим датасет

In [ ]:
!wget https://edunet.kea.su/repo/EduNet-web_dependencies/L02/student_scores.csv
clear_output()

Посмотрим, что там в нем. Видим, что у нас есть два признака - часы и результаты

In [ ]:
import pandas as pd

dataset = pd.read_csv("/content/student_scores.csv")
print(dataset.shape)
dataset.head()

Построим график зависимости одного от другого, а так же отобразим распределения каждой из переменных

In [ ]:
sns.jointplot(data=dataset, x="Scores", y="Hours")
plt.show()

Разделим наши данные на train и test

In [ ]:
from sklearn.model_selection import train_test_split

X = dataset.iloc[:, :-1].values # column Hours
Y = dataset.iloc[:, 1].values # column Score

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

Теперь создадим модель для линейной регрессии. Чтобы не писать с нуля, воспользуемся готовой моделью из библиотеки `sklearn`

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression() 

И обучим ее

In [ ]:
regressor.fit(X_train, Y_train) 

Посмотрим, что получилось

In [ ]:
X_train.shape

In [ ]:
X_points = np.linspace(
    min(X_train), max(X_train), 100
)  # 100 dots at min to max
Y_pred = regressor.predict(X_points)  

plt.figure(figsize=(8, 5))
plt.plot(X_train, Y_train, "o", label="Scores")
plt.plot(X_points, Y_pred, label="y = %.2fx+%.2f" % (regressor.coef_[0], regressor.intercept_ ))
plt.title("Hours vs Percentage", size=15)
plt.xlabel("Hours Studied", size=15)
plt.ylabel("Percentage Score", size=15)
plt.legend()
plt.show()

Теперь сделаем предсказание для тестовой выборки

In [ ]:
Y_pred = regressor.predict(X_test)  

X_points = np.linspace(
    min(X_test), max(X_test), 100
)  
Y_pred = regressor.predict(X_points) 

plt.figure(figsize=(8, 5))
plt.plot(X_test, Y_test, "o", label="Scores")
plt.plot(X_points, Y_pred, label="y = %.2fx+%.2f" % (regressor.coef_[0], regressor.intercept_ ))
plt.title("Hours vs Percentage", size=15)
plt.xlabel("Hours Studied", size=15)
plt.ylabel("Percentage Score", size=15)
plt.legend()
plt.show()

Выглядит не плохо

Посчитаем метрики для наших значений

In [ ]:
from sklearn import metrics

Y_pred = regressor.predict(X_test)

print("Mean Absolute Error: %9.2f" % metrics.mean_absolute_error(Y_test, Y_pred))
print("Mean Squared Error: %10.2f" % metrics.mean_squared_error(Y_test, Y_pred))
print("Root Mean Squared Error: %5.2f" % np.sqrt(metrics.mean_squared_error(Y_test, Y_pred)))

#### Геометрическая интерпретация 

Теперь, когда мы разобрались с тем, что такое регрессия и с чем ее едят, вернемся к нашим картинкам. Как можно применить регрессию для классификации?

Предположим у нас есть только 2 класса. Как можно использовать регрессию для того, чтобы определить относится ли изображение к классу 0 или к классу 1? В упрощенном варианте, задача будет состоять в том, чтобы провести разделяющую плоскость (прямую) между 2-мя классами. Например, мы можем провести прямую через 0. 


<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/regression_for_classification_imgs.png" width="270">





Рассмотрим другую ситуацию, в этом случае, мы не можем просто провести прямую через 0. Но можем отступить от 0 на какое-то расстояние и провести ее там. Вспомним, что уравнение прямой это $y=wx+b$, где $b$ - это смещение (*bias*). Соответственно если b != 0, то прямая через 0 проходить не будет, а будет проходить через значение b.

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/regression_for_classification_add_bias.png" width="270">

[Linear Classification Loss Visualization
](http://vision.stanford.edu/teaching/cs231n-demos/linear-classify/)


Если у нас есть несколько классов (несколько шаблонов), мы можем для каждого из них посчитать уравнение $y_{i} = w_{i}x_{i}+b_{i}$.

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/regression_for_classification_add_bias_add_multiclasses.jpg" width="400">







На картинке нас интересуют 3 класса. Соответственно, мы можем записать систему линейных уравнений:

\begin{aligned}
y_{0} = w_{0}x_{0} + b_{0} \\
y_{1} = w_{1}x_{1} + b_{1} \\
y_{2} = w_{2}x_{2} + b_{2} \\
\end{aligned}

## Метрики регрессии (MSE, MAE, R2)
MSE - повторение мать заикания

## Аналитический расчет производной от функции потерь SVM

### Простые производные 

$$x' = \frac {\delta x} {\delta x} = 1$$ 

$$(x^2)' = \frac {\delta x^2} {\delta x} = 2x$$

$$(\log x)'  = \frac {\delta \log x} {\delta x} = \frac 1 x $$

$$(e^x)'  = \frac {\delta e^x} {\delta x} = e^x $$

$$\frac {\delta cf(x)} {\delta x}= c \cdot \frac {\delta f(x)} {\delta x}$$

$$\frac {\delta f(x) + c} {\delta x}= \frac {\delta f(x)} {\delta x}$$

$c$ - константа, не зависящая от $x$

$$ \frac {\delta [f(x) + g(x)]} {\delta x} = \frac {\delta f(x)} {\delta x}  + \frac {\delta g(x)} {\delta x} $$ 


$$\frac {\delta (x^2 + y^3)} {\delta x} = 2x  $$
так как $y$ по отношению к $x$ - константа и мы меняем только $x$

$$\frac {\delta (x^2 + y^3)} {\delta y} = 3y^2  $$
так как $x$ по отношению к $y$ - константа и мы меняем только $y$

$$(e^y)'  = \frac {\delta e^y} {\delta x} = 0 $$

### Chain-rule

Производная функции $f(g)$:

$$\frac {\delta f} {\delta g}$$

Пусть $g$  на самом деле не просто переменная, а зависит от $h$. Тогда производная от $f$ по $g$ **не меняется**, а производная $f$ по $h$ запишется следующим образом:


$$\frac {\delta f(g(h))} {\delta h} = \frac {\delta f} {\delta g} \frac {\delta g} {\delta h}$$

Пусть теперь $h$ зависит от $x$. Все аналогично


$$\frac {\delta f(g(h(x)))} {\delta x} = \frac {\delta f} {\delta g} \frac {\delta g} {\delta h} \frac {\delta h} {\delta x}$$

Так можно делать до бесконечности, находя производную сколь угодно сложной функции. И, что важно - мы можем считать градиенты частями - посчитать сначала $f$ по $g$, потом $g$ по $h$....

$$\frac {\delta log(x^2 + 5)} {\delta x}$$

$$h = x^2 + 5$$

$$\frac {\delta log(x^2 + 5)} {\delta x} = \frac {\delta log(h)} {\delta h} \frac {\delta h} {\delta x}$$ 

$$\frac {\delta log(h)} {\delta h} = \frac 1 h$$

$$\frac  {\delta h} {\delta x} = 2x $$

$$\frac {\delta log(x^2 + 5)} {\delta x} = \frac 1 {x^2 + 5} \cdot 2x = \frac {2x} {x^2 + 5}$$


###  Часть MSE-loss

$$loss = (y - \hat{y})^2 $$

$$\hat{y} = wx + b $$

$$ \frac {\delta loss} {\delta w} = \frac {\delta loss} {\delta \hat{y}} \cdot \frac {\delta \hat{y}} {\delta w} $$

$$ \frac {\delta loss} {\delta \hat{y}} = \frac {\delta (y - \hat{y})^2 } {\delta {(y - \hat{y})}} \frac {\delta y - \hat{y}} {\delta \hat{y}} = 2(y-\hat{y}) \cdot -1 = 2 (\hat{y} - y)$$

$$ \frac {\delta \hat{y}} {\delta w} = \frac {\delta wx + b} {\delta w} = x$$

$$ \frac {\delta loss} {\delta w} = 2 x \cdot (\hat{y} - y) $$

### MSE-loss
$$MSE = \frac 1 N \sum_i(y_i - \hat{y_i})^2 $$

$y_i$ - константы
$\hat{y_i}$ - не являются функциями друг от друга 
$$\hat{y} = wx_i + b $$

$$\frac {\delta MSE} {\delta w} = \frac 1 N \sum \frac {\delta (y_i - \hat{y_i}) ^2} {\delta \hat{y_i}} \frac {\delta \hat{y_i}} {\delta w}$$




### Часть MAE-Loss

$$loss = |y - \hat{y}| $$

$$\hat{y} = wx + b $$

$$ \frac {\delta loss} {\delta w} = \frac {\delta loss} {\delta \hat{y}} \cdot \frac {\delta \hat{y}} {\delta w} $$

$$ \frac {\delta \hat{y}} {\delta w} = \frac {\delta wx + b} {\delta w} = x$$

$$ \frac {\delta loss} {\delta \hat{y}} = \frac {\delta |y - \hat{y}|) } {\delta {(y - \hat{y})}} \frac {\delta y - \hat{y}} {\delta \hat{y}} = \frac {\delta |y - \hat{y}|) } {\delta {(y - \hat{y})}} \cdot -1 = - \frac {\delta |y - \hat{y}|) } {\delta {(y - \hat{y})}}$$

Строго говоря, у модуля не существует производной в 0. 







In [ ]:
import matplotlib.pyplot as plt

X = [i for i in range(-5, 6)]
Y = [abs(i) for i in range(-5, 6)]

plt.figure(figsize=(8, 5))
plt.plot(X, Y, label="y = |x|")
plt.title("y = |x|", size=20)
plt.legend()
plt.show()

Но мы можем сказать, что в этой точке производная равна 0.
Если аргумент модуля меньше 0, то производная будет -1.

Если больше +1

$$ \frac {\delta loss} {\delta \hat{y}} = \frac {\delta |y - \hat{y}| } {\delta {(y - \hat{y})}} \frac {\delta y - \hat{y}} {\delta \hat{y}} = \frac {\delta |y - \hat{y}| } {\delta {(y - \hat{y})}} \cdot -1 = - \frac {\delta |y - \hat{y}| } {\delta {(y - \hat{y})}} = - sign(y - \hat{y}) =  sign(\hat{y} - y)$$

<br>
<br>


In [ ]:
X = [i for i in range(-5, 1, 1)]
Y = [i * 0 - 1 for i in range(6)]
X_1 = [i for i in range(0, 6)]
Y_1 = [i * 0 + 1 for i in range(0, 6)]

plt.figure(figsize=(8, 5))
plt.plot(X, Y, "b")
plt.plot(X_1, Y_1, "b")
plt.plot(0, 0, "ro")
plt.plot(0, 1, "bo")
plt.plot(0, -1, "bo")
plt.show()

### Max-Loss

$$b = max(x, y)$$

$$b = x~~if~~x > y~~else~~y$$


$$\frac {\delta b} {\delta x} =  \frac {\delta x} {\delta x}~~if~~x > y~~else~~\frac {\delta y} {\delta x} = 1~~if~~x > y~~else~~0$$

Если $x > y$, то он оказал влияние на $b$. Иначе, его вклада в $b$ НЕ БЫЛО - градиент равен 0

 


### SVM-Loss

Из: 

$L_i=\sum_{j\neq y_i}max(0,s_j-s_{y_i}+1)$

Получаем:

$ \nabla_W	L(W) = {1 \over N}\sum_{i=1}^N \nabla_W L_i(x_i, y_i, W)$

##### $\color{blue}{\text{*Не обязательное задание:}}$

Поставлю дополнительно 20 баллов тому, кто графически оформит результаты этого эксперимента и напишет вывод о зависимости качества обучения от learning_rate и batch_size (возможно, нужно будет добавить варианты их сочетаний и/или увеличить количество эпох до 20).

In [ ]:
import os
from IPython.display import clear_output

file_exists = os.path.exists("/content/cifar-10-batches-py")
if file_exists == False:
    !wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
    !tar -xzf cifar-10-python.tar.gz   
clear_output()

In [ ]:
import numpy as np
import pickle

def unpickle(file):
    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict


X_train = np.zeros((0, 3072))
Y_train = np.array([])
for i in range(1, 6):
    raw = unpickle(f"/content/cifar-10-batches-py/data_batch_{i}")
    X_train = np.append(X_train, np.array(raw[b"data"]), axis=0)
    Y_train = np.append(Y_train, np.array(raw[b"labels"]), axis=0)

test = unpickle("/content/cifar-10-batches-py/test_batch")
X_test = np.array(test[b"data"])
Y_test = np.array(test[b"labels"])

labels_eng = [
    "Airplane",
    "Car",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Ship",
    "Truck",
]

print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_train shape: {X_test.shape}, Y_train shape: {Y_test.shape}")

In [ ]:
import random

class LinearClassifier():
    def __init__(self, labels, batch_size, random_state=42):
        self.labels = labels  # classes names
        self.classes_num = len(labels)  # num of classes

        np.random.seed(
            random_state
        )  
        self.W = (
            np.random.randn(3073, self.classes_num) * 0.0001
        )  # generate random weights, reshape to add bias 
        self.batch_size = batch_size  # batch_size

    def fit(self, X_train, Y_train, learning_rate=1e-8):
        loss = 0.0  # обнуляем loss
        train_len = X_train.shape[0]  # num of examples
        indexes = list(range(train_len))  # indexes train_len
        random.shuffle(indexes)  

        for i in range(
            0, train_len, self.batch_size
        ):  
            idx = indexes[
                i : i + self.batch_size
            ]  # 
            X_batch = X_train[idx]  
            Y_batch = Y_train[idx]  

            X_batch = np.hstack(
                [X_batch, np.ones((X_batch.shape[0], 1))]
            )  # add bias

            loss_val, grad = self.loss(X_batch, Y_batch)  # loss and gradient
            self.W -= learning_rate * grad  # update weigths

            loss += loss_val  # loss sum
        return loss / (train_len)  # mean loss

    def loss(self, X, Y):
        current_batch_size = X.shape[0]  # batch_size
        loss = 0.0  
        dW = np.zeros(self.W.shape) 
        for i in range(current_batch_size):  
            scores = X[i].dot(
                self.W
            )  # vector of shape 10
            correct_class_score = scores[
                int(Y[i])
            ]  
            above_zero_loss_count = 0  
            for j in range(self.classes_num): 
                if j == Y[i]:  # predict class
                    continue
                margin = scores[j] - correct_class_score + 1  # loss
                if margin > 0:  
                    above_zero_loss_count += (
                        1  
                    )
                    loss += margin  # 
                    dW[:, j] += X[i]  # 
            dW[:, int(Y[i])] -= above_zero_loss_count * X[i]  
        loss /= current_batch_size  
        dW /= current_batch_size  
        return loss, dW

    def forward(self, X):
        X = np.append(X, 1)  # add 1 (bias)
        scores = X.dot(self.W)  
        return np.argmax(scores)  

In [ ]:

def validate(model, X_test, Y_test, noprint=False):
    correct = 0  
    for i, img in enumerate(X_test):  
        index = model.forward(img)  
        correct += (
            1 if index == Y_test[i] else 0
        )  
        if noprint is False:  
            if i > 0 and i % 1000 == 0:  
                print(
                    "Accuracy {:.3f}".format(correct / i)
                )  
    return correct / len(Y_test)  

In [ ]:
print("How learning quality depends of speed:")

for lr in [1e-2, 1e-8]:
    for bs in [256, 2048]:

        print("-" * 50, "\n", "learning_rate =", lr, "\tbatch_size =", bs)
        print()
        lc_model = LinearClassifier(labels_eng, batch_size=bs)

        best_accuracy = 0
        for epoch in range(10):
            loss = lc_model.fit(X_train, Y_train, learning_rate=lr)
            accuracy = validate(lc_model, X_test, Y_test, noprint=True)
            if best_accuracy < accuracy:
                best_accuracy = accuracy
                best_epoch = epoch
            print(f"Epoch {epoch} \tLoss: {loss}, \tAccuracy:{accuracy}")

        print()
        print(f"Best accuracy is {best_accuracy} in {best_epoch} epoch")

## Bias-variance tradeoff на примере линейной регрессии и KNN
(просто переписать все)


### Bias, Variance, Irreducible error 

Можно показать, что ошибка любой модели раскладывается в сумму трех компонент:

$$ Model\_error = Bias^2 + Variance + Irreducible\_error $$

#### Bias<a class="anchor" style="autocontent" id="Bias"/><br>
Обычно, высокий bias имеют недообученные модели. Например, реальная зависимость, которую мы наблюдаем - нелинейная, а мы пытаемся аппроксимировать ее линией. В этом случае наше решение заведомо смещено (biassed) в сторону линейной модели и мы всегда будем ошибаться в сравнении с реальной моделью данных

<img src ="https://edunet.kea.su/repo/EduNet-content/L03/img_license/problem_of_hight_bias.png" width="700">

#### Variance
Можно получить и обратную ситуацию. Реальная закономерность имеет линейный вид, а мы пытаемся ее аппроксимировать нелинейной моделью. 
В этом случае мы будем выучивать любой шум в данных и пытаться объяснить его нашей моделью. 
Малое изменение в данных будет приводить к большим изменениям в прогнозе модели. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L03/img_license/problem_of_hight_variance.png" width="700">

Иногда bias и variance представляет еще таким образом:
1. можно быть очень точным и попадать всегда в центр мишени - это соответствует низкому bias и низкому variance;
2. можно попадать примерно в центр мишени, но при этом с большим разбросом - низкий bias, но высокий variance;
3. можно стрелять кучно, но не туда) - это высокий bias и низкий variance;
4. ну а можно просто стрелять наугад, куда душа зовет - это высокий bias и высокий variance.



<img src ="https://edunet.kea.su/repo/EduNet-content/L03/img_license/low_hight_bias_variance.png" width="450">

#### Irreducible error<a class="anchor" style="autocontent" id="Irreducible-error"/><br>
В идеальном для нас случае - когда мы угадали с моделью наших данных, гипотетически можно получить. $$Bias=0, Variance=0$$
Однако, у нас есть ошибки в измерении самой предсказываемой величины. Из-за этого наша модель всегда будет иметь некий уровень ошибки, ниже которого опуститься нельзя. 




#### Bias vs variance<a class="anchor" style="autocontent" id="Bias-vs-variance"/><br>

В реальности же, когда реальную модель данных угадать в точности почти невозможно, есть bias-variance tradeoff - нельзя бесконечно уменьшать и Bias, и Variance. Есть какая-то точка оптимума. С какого-то момента при уменьшении Bias начнет увеличиваться Variance, и наоборот. 
При этом, можно построить  связь этих величин с увеличением сложности модели (capacity)

<img src ="https://edunet.kea.su/repo/EduNet-content/L03/img_license/bias_variance_tradeoff.png" width="450">


#### Применительно к деревьям<a class="anchor" style="autocontent" id="Применительно-к-деревьям"/><br>
Дерева малой глубины имеет малую сложность - и высокий bias. 
Дерево большой глубины имеет высокую сложность - и высокий variance. 

Можно подобрать для дерева идеальную capacity, когда Bias и Variance будут суммарно давать наименьший вклад в ошибку. Этим мы занимаемся при подборе параметров. 
Но, оказывается, есть и другие способы борьбы с variance и/или bias, которые мы разберем позже.

Заметим, что если бы могли говорить не просто решение дерева, а привязывать к этому какую-то статистику, например, сколько деревьев, построенных по подобной процедуре, приняли такое же решение - было бы легче.

Если наложить решающие границы 100 решающих деревьев, построенных на разных выборках из X, y, то мы увидим, что "хорошие области", соответствующие реальному разделению данных будут общими между деревьями, а плохие - индивидуальны. 
К сожалению, в реальности, мы не можем брать бесконечное число наборов данных из генеральной совокупности (представленной в данном случае X, y)


In [ ]:
plt.figure(figsize=(6,6))
 
for i in range(1,101):
    X_train, X_test, Y_train, Y_test = train_test_split( X, Y, random_state = i)
    clf = DecisionTreeClassifier(max_depth = 20, random_state = 0)
    clf.fit(X_train, Y_train)
    plot_decision_boundary(clf, X, Y, alpha = 0.02, contour=False)

plt.show()

# Классификация

## Возврат к классификации






## SVM

# Функция потерь

## на примере SVM лосс

### Аналитический расчет производной от функции потерь SVM

### Простые производные 

$$x' = \frac {\delta x} {\delta x} = 1$$ 

$$(x^2)' = \frac {\delta x^2} {\delta x} = 2x$$

$$(\log x)'  = \frac {\delta \log x} {\delta x} = \frac 1 x $$

$$(e^x)'  = \frac {\delta e^x} {\delta x} = e^x $$

$$\frac {\delta cf(x)} {\delta x}= c \cdot \frac {\delta f(x)} {\delta x}$$

$$\frac {\delta f(x) + c} {\delta x}= \frac {\delta f(x)} {\delta x}$$

$c$ - константа, не зависящая от $x$

$$ \frac {\delta [f(x) + g(x)]} {\delta x} = \frac {\delta f(x)} {\delta x}  + \frac {\delta g(x)} {\delta x} $$ 


$$\frac {\delta (x^2 + y^3)} {\delta x} = 2x  $$
так как $y$ по отношению к $x$ - константа и мы меняем только $x$

$$\frac {\delta (x^2 + y^3)} {\delta y} = 3y^2  $$
так как $x$ по отношению к $y$ - константа и мы меняем только $y$

$$(e^y)'  = \frac {\delta e^y} {\delta x} = 0 $$

### Chain-rule

Производная функции $f(g)$:

$$\frac {\delta f} {\delta g}$$

Пусть $g$  на самом деле не просто переменная, а зависит от $h$. Тогда производная от $f$ по $g$ **не меняется**, а производная $f$ по $h$ запишется следующим образом:


$$\frac {\delta f(g(h))} {\delta h} = \frac {\delta f} {\delta g} \frac {\delta g} {\delta h}$$

Пусть теперь $h$ зависит от $x$. Все аналогично


$$\frac {\delta f(g(h(x)))} {\delta x} = \frac {\delta f} {\delta g} \frac {\delta g} {\delta h} \frac {\delta h} {\delta x}$$

Так можно делать до бесконечности, находя производную сколь угодно сложной функции. И, что важно - мы можем считать градиенты частями - посчитать сначала $f$ по $g$, потом $g$ по $h$....

$$\frac {\delta log(x^2 + 5)} {\delta x}$$

$$h = x^2 + 5$$

$$\frac {\delta log(x^2 + 5)} {\delta x} = \frac {\delta log(h)} {\delta h} \frac {\delta h} {\delta x}$$ 

$$\frac {\delta log(h)} {\delta h} = \frac 1 h$$

$$\frac  {\delta h} {\delta x} = 2x $$

$$\frac {\delta log(x^2 + 5)} {\delta x} = \frac 1 {x^2 + 5} \cdot 2x = \frac {2x} {x^2 + 5}$$


###  Часть MSE-loss

$$loss = (y - \hat{y})^2 $$

$$\hat{y} = wx + b $$

$$ \frac {\delta loss} {\delta w} = \frac {\delta loss} {\delta \hat{y}} \cdot \frac {\delta \hat{y}} {\delta w} $$

$$ \frac {\delta loss} {\delta \hat{y}} = \frac {\delta (y - \hat{y})^2 } {\delta {(y - \hat{y})}} \frac {\delta y - \hat{y}} {\delta \hat{y}} = 2(y-\hat{y}) \cdot -1 = 2 (\hat{y} - y)$$

$$ \frac {\delta \hat{y}} {\delta w} = \frac {\delta wx + b} {\delta w} = x$$

$$ \frac {\delta loss} {\delta w} = 2 x \cdot (\hat{y} - y) $$

### MSE-loss
$$MSE = \frac 1 N \sum_i(y_i - \hat{y_i})^2 $$

$y_i$ - константы
$\hat{y_i}$ - не являются функциями друг от друга 
$$\hat{y} = wx_i + b $$

$$\frac {\delta MSE} {\delta w} = \frac 1 N \sum \frac {\delta (y_i - \hat{y_i}) ^2} {\delta \hat{y_i}} \frac {\delta \hat{y_i}} {\delta w}$$




### Часть MAE-Loss

$$loss = |y - \hat{y}| $$

$$\hat{y} = wx + b $$

$$ \frac {\delta loss} {\delta w} = \frac {\delta loss} {\delta \hat{y}} \cdot \frac {\delta \hat{y}} {\delta w} $$

$$ \frac {\delta \hat{y}} {\delta w} = \frac {\delta wx + b} {\delta w} = x$$

$$ \frac {\delta loss} {\delta \hat{y}} = \frac {\delta |y - \hat{y}|) } {\delta {(y - \hat{y})}} \frac {\delta y - \hat{y}} {\delta \hat{y}} = \frac {\delta |y - \hat{y}|) } {\delta {(y - \hat{y})}} \cdot -1 = - \frac {\delta |y - \hat{y}|) } {\delta {(y - \hat{y})}}$$

Строго говоря, у модуля не существует производной в 0. 







In [ ]:
import matplotlib.pyplot as plt

X = [i for i in range(-5, 6)]
Y = [abs(i) for i in range(-5, 6)]

plt.figure(figsize=(8, 5))
plt.plot(X, Y, label="y = |x|")
plt.title("y = |x|", size=20)
plt.legend()
plt.show()

Но мы можем сказать, что в этой точке производная равна 0.
Если аргумент модуля меньше 0, то производная будет -1.

Если больше +1

$$ \frac {\delta loss} {\delta \hat{y}} = \frac {\delta |y - \hat{y}| } {\delta {(y - \hat{y})}} \frac {\delta y - \hat{y}} {\delta \hat{y}} = \frac {\delta |y - \hat{y}| } {\delta {(y - \hat{y})}} \cdot -1 = - \frac {\delta |y - \hat{y}| } {\delta {(y - \hat{y})}} = - sign(y - \hat{y}) =  sign(\hat{y} - y)$$

<br>
<br>


In [ ]:
X = [i for i in range(-5, 1, 1)]
Y = [i * 0 - 1 for i in range(6)]
X_1 = [i for i in range(0, 6)]
Y_1 = [i * 0 + 1 for i in range(0, 6)]

plt.figure(figsize=(8, 5))
plt.plot(X, Y, "b")
plt.plot(X_1, Y_1, "b")
plt.plot(0, 0, "ro")
plt.plot(0, 1, "bo")
plt.plot(0, -1, "bo")
plt.show()

### Max-Loss

$$b = max(x, y)$$

$$b = x~~if~~x > y~~else~~y$$


$$\frac {\delta b} {\delta x} =  \frac {\delta x} {\delta x}~~if~~x > y~~else~~\frac {\delta y} {\delta x} = 1~~if~~x > y~~else~~0$$

Если $x > y$, то он оказал влияние на $b$. Иначе, его вклада в $b$ НЕ БЫЛО - градиент равен 0

 


### SVM-Loss

Из: 

$L_i=\sum_{j\neq y_i}max(0,s_j-s_{y_i}+1)$

Получаем:

$ \nabla_W	L(W) = {1 \over N}\sum_{i=1}^N \nabla_W L_i(x_i, y_i, W)$

##### $\color{blue}{\text{*Не обязательное задание:}}$

Поставлю дополнительно 20 баллов тому, кто графически оформит результаты этого эксперимента и напишет вывод о зависимости качества обучения от learning_rate и batch_size (возможно, нужно будет добавить варианты их сочетаний и/или увеличить количество эпох до 20).

In [ ]:
import os
from IPython.display import clear_output

file_exists = os.path.exists("/content/cifar-10-batches-py")
if file_exists == False:
    !wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
    !tar -xzf cifar-10-python.tar.gz   
clear_output()

In [ ]:
import numpy as np
import pickle

def unpickle(file):
    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict


X_train = np.zeros((0, 3072))
Y_train = np.array([])
for i in range(1, 6):
    raw = unpickle(f"/content/cifar-10-batches-py/data_batch_{i}")
    X_train = np.append(X_train, np.array(raw[b"data"]), axis=0)
    Y_train = np.append(Y_train, np.array(raw[b"labels"]), axis=0)

test = unpickle("/content/cifar-10-batches-py/test_batch")
X_test = np.array(test[b"data"])
Y_test = np.array(test[b"labels"])

labels_eng = [
    "Airplane",
    "Car",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Ship",
    "Truck",
]

print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_train shape: {X_test.shape}, Y_train shape: {Y_test.shape}")

In [ ]:
import random

class LinearClassifier():
    def __init__(self, labels, batch_size, random_state=42):
        self.labels = labels  # classes names
        self.classes_num = len(labels)  # num of classes

        np.random.seed(
            random_state
        )  
        self.W = (
            np.random.randn(3073, self.classes_num) * 0.0001
        )  # generate random weights, reshape to add bias 
        self.batch_size = batch_size  # batch_size

    def fit(self, X_train, Y_train, learning_rate=1e-8):
        loss = 0.0  # обнуляем loss
        train_len = X_train.shape[0]  # num of examples
        indexes = list(range(train_len))  # indexes train_len
        random.shuffle(indexes)  

        for i in range(
            0, train_len, self.batch_size
        ):  
            idx = indexes[
                i : i + self.batch_size
            ]  # 
            X_batch = X_train[idx]  
            Y_batch = Y_train[idx]  

            X_batch = np.hstack(
                [X_batch, np.ones((X_batch.shape[0], 1))]
            )  # add bias

            loss_val, grad = self.loss(X_batch, Y_batch)  # loss and gradient
            self.W -= learning_rate * grad  # update weigths

            loss += loss_val  # loss sum
        return loss / (train_len)  # mean loss

    def loss(self, X, Y):
        current_batch_size = X.shape[0]  # batch_size
        loss = 0.0  
        dW = np.zeros(self.W.shape) 
        for i in range(current_batch_size):  
            scores = X[i].dot(
                self.W
            )  # vector of shape 10
            correct_class_score = scores[
                int(Y[i])
            ]  
            above_zero_loss_count = 0  
            for j in range(self.classes_num): 
                if j == Y[i]:  # predict class
                    continue
                margin = scores[j] - correct_class_score + 1  # loss
                if margin > 0:  
                    above_zero_loss_count += (
                        1  
                    )
                    loss += margin  # 
                    dW[:, j] += X[i]  # 
            dW[:, int(Y[i])] -= above_zero_loss_count * X[i]  
        loss /= current_batch_size  
        dW /= current_batch_size  
        return loss, dW

    def forward(self, X):
        X = np.append(X, 1)  # add 1 (bias)
        scores = X.dot(self.W)  
        return np.argmax(scores)  

In [ ]:

def validate(model, X_test, Y_test, noprint=False):
    correct = 0  
    for i, img in enumerate(X_test):  
        index = model.forward(img)  
        correct += (
            1 if index == Y_test[i] else 0
        )  
        if noprint is False:  
            if i > 0 and i % 1000 == 0:  
                print(
                    "Accuracy {:.3f}".format(correct / i)
                )  
    return correct / len(Y_test)  

In [ ]:
print("How learning quality depends of speed:")

for lr in [1e-2, 1e-8]:
    for bs in [256, 2048]:

        print("-" * 50, "\n", "learning_rate =", lr, "\tbatch_size =", bs)
        print()
        lc_model = LinearClassifier(labels_eng, batch_size=bs)

        best_accuracy = 0
        for epoch in range(10):
            loss = lc_model.fit(X_train, Y_train, learning_rate=lr)
            accuracy = validate(lc_model, X_test, Y_test, noprint=True)
            if best_accuracy < accuracy:
                best_accuracy = accuracy
                best_epoch = epoch
            print(f"Epoch {epoch} \tLoss: {loss}, \tAccuracy:{accuracy}")

        print()
        print(f"Best accuracy is {best_accuracy} in {best_epoch} epoch")

# Градиент

# Обновления весов методом градиентного спуска

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/backpropagation_weight_optimization.png" width="750">

Идея в следующем: у нас есть некоторая поверхность, на которой будет точка минимума функции. Мы должны обновлять веса таким образом, чтобы смещаться в сторону этой точки. Loss показывает, как возрастает функция потерь. Так как потери - это плохо, мы должны их минимизировать, то есть мы должны уменьшать веса в сторону, обратную росту этой функции.

Если переходить на случай n-мерного (в данном случае трехмерного) пространства, здесь достаточно очевидна аналогия с поверхностью: у нас есть поверхность земли, которая описывается координатами $x, y, z$. Если мы движемся по этой поверхности, высота $(z)$ будет зависеть от $x, y$. 

$x, y$ - это координаты. Мы можем записать их как вектор. Наша функция будет работать с вектором координат и выдавать скаляр, третью координату. Если в качестве координат мы будем использовать веса нашей модели, а в качестве $z$ - loss, то аналогия станет полной. Наша задача сведется к тому, чтобы найти такой набор весов, при котором значение функции будет минимально. То есть мы окажемся в каком-то минимуме, где ошибка будет минимально возможна для этих данных.

Для того чтобы предположить, где он может находиться, надо понимать, в какую сторону функция растет, а в какую - убывает. Для этого существует производная.

Поскольку у нас функция от нескольких переменных, если мы будем брать от нее производную, у нас получится вектор частных производных, то есть градиент этой функции, который будет показывать, как она меняется в каждом направлении.

## Градиент функции потерь


Градиент — вектор, своим направлением указывающий направление наибольшего возрастания некоторой величины **φ**, значение которой меняется от одной точки пространства к другой (скалярного поля), а по величине (модулю) равный скорости роста этой величины в этом направлении.

Например, если взять в качестве **φ**  высоту поверхности земли над уровнем моря, то её градиент в каждой точке поверхности будет показывать «направление самого крутого подъёма», и своей величиной характеризовать крутизну склона.

Другими словами, градиент — это производная по пространству, но в отличие от производной по одномерному времени, градиент является не скаляром, а векторной величиной.

Для случая трёхмерного пространства градиентом скалярной функции **φ=φ(x,y,z)** координат **(x,y,z)** называется векторная функция с компонентами
$$\frac{\partial\varphi}{\partial x}, \frac{\partial\varphi}{\partial y}, \frac{\partial\varphi}{\partial z}$$.

Если **φ**  — функция **n** переменных **X1...xn**, то её градиентом называется **n**-мерный вектор,
$$\frac{\partial\varphi}{\partial x_1},...,\frac{\partial\varphi}{\partial x_n}$$



компоненты которого, равны частным производным **φ** по всем её аргументам.

Размерность вектора градиента определяется размерностью пространства (или многообразия), на котором задано скалярное поле, о градиенте которого идёт речь.
Оператором градиента называется оператор, действие которого на скалярную функцию (поле) даёт её градиент. Этого оператора иногда коротко называют просто «градиентом».


$W$ - матрица(вектор) весов

$L$ - функция потерь

$\partial W = W_2 - W_1$

$\partial L = L_2 - L_1$


$\frac{\partial L}{\partial W}=\begin{bmatrix}
\frac{\partial L}{\partial W_1} \\
\frac{\partial L}{\partial W_2} \\
... \\
\frac{\partial L}{\partial W_n}
\end{bmatrix}$


Наша задача будет сводиться к тому, что мы будем искать градиент loss функции по весам, которые будут состоять из производной по каждому направлению. Поскольку у нас здесь числа, можно считать этот градиент численно.

## Численный расчет производной

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/gradient_descent_analytical_calculation.png" width="650">



Посчитаем градиент приближенно, воспользовавшись определением (в формуле аргумент обозначен как $x$, у нас же аргументом будет $W$):
На нулевом шаге у нас есть $W_0$ найдем $L_0 = Loss(f(W_0,x))$
Прибавим к первому элементу $W_0$ небольшую величину  $h$ = 0.0001 и получим новую матрицу весов $W_1$ отличающуюся от $W_0$ на один единственный элемент.

Найдем Loss от $\frac {W_1}  {L_1} = Loss(f(W_1,x))$
По определению производной $\frac {dL}{W_0} = \frac{( L_1 - L_0 )}   {h}$

Повторяя этот процесс для каждого элемента из $W$, найдем вектор частных производных, то есть градиент $\frac{dL}{dW}$.

Плюсы:
Это просто. 

Проблемы:

1. Это очень долго, нам придется заново искать значение loss функции для каждого $W_i$.

2. Это неточно, так как по определению приращение $h$ бесконечно мало, а мы используем конкретное пусть и небольшое число. И если мы сделаем его слишком маленьким, то столкнемся с ошибками связанными с округлением в памяти компьютера.


Поэтому данный метод может использоваться как проверочный. 
А на практике вместо него используется **аналитический расчет градиента**.


## Аналитический расчет производной от функции потерь SVM

### Простые производные 

$$x' = \frac {\delta x} {\delta x} = 1$$ 

$$(x^2)' = \frac {\delta x^2} {\delta x} = 2x$$

$$(\log x)'  = \frac {\delta \log x} {\delta x} = \frac 1 x $$

$$(e^x)'  = \frac {\delta e^x} {\delta x} = e^x $$

$$\frac {\delta cf(x)} {\delta x}= c \cdot \frac {\delta f(x)} {\delta x}$$

$$\frac {\delta f(x) + c} {\delta x}= \frac {\delta f(x)} {\delta x}$$

$c$ - константа, не зависящая от $x$

$$ \frac {\delta [f(x) + g(x)]} {\delta x} = \frac {\delta f(x)} {\delta x}  + \frac {\delta g(x)} {\delta x} $$ 


$$\frac {\delta (x^2 + y^3)} {\delta x} = 2x  $$
так как $y$ по отношению к $x$ - константа и мы меняем только $x$

$$\frac {\delta (x^2 + y^3)} {\delta y} = 3y^2  $$
так как $x$ по отношению к $y$ - константа и мы меняем только $y$

$$(e^y)'  = \frac {\delta e^y} {\delta x} = 0 $$

### Chain-rule

Производная функции $f(g)$:

$$\frac {\delta f} {\delta g}$$

Пусть $g$  на самом деле не просто переменная, а зависит от $h$. Тогда производная от $f$ по $g$ **не меняется**, а производная $f$ по $h$ запишется следующим образом:


$$\frac {\delta f(g(h))} {\delta h} = \frac {\delta f} {\delta g} \frac {\delta g} {\delta h}$$

Пусть теперь $h$ зависит от $x$. Все аналогично


$$\frac {\delta f(g(h(x)))} {\delta x} = \frac {\delta f} {\delta g} \frac {\delta g} {\delta h} \frac {\delta h} {\delta x}$$

Так можно делать до бесконечности, находя производную сколь угодно сложной функции. И, что важно - мы можем считать градиенты частями - посчитать сначала $f$ по $g$, потом $g$ по $h$....

$$\frac {\delta log(x^2 + 5)} {\delta x}$$

$$h = x^2 + 5$$

$$\frac {\delta log(x^2 + 5)} {\delta x} = \frac {\delta log(h)} {\delta h} \frac {\delta h} {\delta x}$$ 

$$\frac {\delta log(h)} {\delta h} = \frac 1 h$$

$$\frac  {\delta h} {\delta x} = 2x $$

$$\frac {\delta log(x^2 + 5)} {\delta x} = \frac 1 {x^2 + 5} \cdot 2x = \frac {2x} {x^2 + 5}$$


###  Часть MSE-loss

$$loss = (y - \hat{y})^2 $$

$$\hat{y} = wx + b $$

$$ \frac {\delta loss} {\delta w} = \frac {\delta loss} {\delta \hat{y}} \cdot \frac {\delta \hat{y}} {\delta w} $$

$$ \frac {\delta loss} {\delta \hat{y}} = \frac {\delta (y - \hat{y})^2 } {\delta {(y - \hat{y})}} \frac {\delta y - \hat{y}} {\delta \hat{y}} = 2(y-\hat{y}) \cdot -1 = 2 (\hat{y} - y)$$

$$ \frac {\delta \hat{y}} {\delta w} = \frac {\delta wx + b} {\delta w} = x$$

$$ \frac {\delta loss} {\delta w} = 2 x \cdot (\hat{y} - y) $$

### MSE-loss
$$MSE = \frac 1 N \sum_i(y_i - \hat{y_i})^2 $$

$y_i$ - константы
$\hat{y_i}$ - не являются функциями друг от друга 
$$\hat{y} = wx_i + b $$

$$\frac {\delta MSE} {\delta w} = \frac 1 N \sum \frac {\delta (y_i - \hat{y_i}) ^2} {\delta \hat{y_i}} \frac {\delta \hat{y_i}} {\delta w}$$




### Часть MAE-Loss

$$loss = |y - \hat{y}| $$

$$\hat{y} = wx + b $$

$$ \frac {\delta loss} {\delta w} = \frac {\delta loss} {\delta \hat{y}} \cdot \frac {\delta \hat{y}} {\delta w} $$

$$ \frac {\delta \hat{y}} {\delta w} = \frac {\delta wx + b} {\delta w} = x$$

$$ \frac {\delta loss} {\delta \hat{y}} = \frac {\delta |y - \hat{y}|) } {\delta {(y - \hat{y})}} \frac {\delta y - \hat{y}} {\delta \hat{y}} = \frac {\delta |y - \hat{y}|) } {\delta {(y - \hat{y})}} \cdot -1 = - \frac {\delta |y - \hat{y}|) } {\delta {(y - \hat{y})}}$$

Строго говоря, у модуля не существует производной в 0. 







In [ ]:
import matplotlib.pyplot as plt

X = [i for i in range(-5, 6)]
Y = [abs(i) for i in range(-5, 6)]

plt.figure(figsize=(8, 5))
plt.plot(X, Y, label="y = |x|")
plt.title("y = |x|", size=20)
plt.legend()
plt.show()

Но мы можем сказать, что в этой точке производная равна 0.
Если аргумент модуля меньше 0, то производная будет -1.

Если больше +1

$$ \frac {\delta loss} {\delta \hat{y}} = \frac {\delta |y - \hat{y}| } {\delta {(y - \hat{y})}} \frac {\delta y - \hat{y}} {\delta \hat{y}} = \frac {\delta |y - \hat{y}| } {\delta {(y - \hat{y})}} \cdot -1 = - \frac {\delta |y - \hat{y}| } {\delta {(y - \hat{y})}} = - sign(y - \hat{y}) =  sign(\hat{y} - y)$$

<br>
<br>


In [ ]:
X = [i for i in range(-5, 1, 1)]
Y = [i * 0 - 1 for i in range(6)]
X_1 = [i for i in range(0, 6)]
Y_1 = [i * 0 + 1 for i in range(0, 6)]

plt.figure(figsize=(8, 5))
plt.plot(X, Y, "b")
plt.plot(X_1, Y_1, "b")
plt.plot(0, 0, "ro")
plt.plot(0, 1, "bo")
plt.plot(0, -1, "bo")
plt.show()

### Max-Loss

$$b = max(x, y)$$

$$b = x~~if~~x > y~~else~~y$$


$$\frac {\delta b} {\delta x} =  \frac {\delta x} {\delta x}~~if~~x > y~~else~~\frac {\delta y} {\delta x} = 1~~if~~x > y~~else~~0$$

Если $x > y$, то он оказал влияние на $b$. Иначе, его вклада в $b$ НЕ БЫЛО - градиент равен 0

 


### SVM-Loss

Из: 

$L_i=\sum_{j\neq y_i}max(0,s_j-s_{y_i}+1)$

Получаем:

$ \nabla_W	L(W) = {1 \over N}\sum_{i=1}^N \nabla_W L_i(x_i, y_i, W)$

##### $\color{blue}{\text{*Не обязательное задание:}}$

Поставлю дополнительно 20 баллов тому, кто графически оформит результаты этого эксперимента и напишет вывод о зависимости качества обучения от learning_rate и batch_size (возможно, нужно будет добавить варианты их сочетаний и/или увеличить количество эпох до 20).

In [ ]:
import os
from IPython.display import clear_output

file_exists = os.path.exists("/content/cifar-10-batches-py")
if file_exists == False:
    !wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
    !tar -xzf cifar-10-python.tar.gz   
clear_output()

In [ ]:
import numpy as np
import pickle

def unpickle(file):
    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict


X_train = np.zeros((0, 3072))
Y_train = np.array([])
for i in range(1, 6):
    raw = unpickle(f"/content/cifar-10-batches-py/data_batch_{i}")
    X_train = np.append(X_train, np.array(raw[b"data"]), axis=0)
    Y_train = np.append(Y_train, np.array(raw[b"labels"]), axis=0)

test = unpickle("/content/cifar-10-batches-py/test_batch")
X_test = np.array(test[b"data"])
Y_test = np.array(test[b"labels"])

labels_eng = [
    "Airplane",
    "Car",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Ship",
    "Truck",
]

print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_train shape: {X_test.shape}, Y_train shape: {Y_test.shape}")

In [ ]:
import random

class LinearClassifier():
    def __init__(self, labels, batch_size, random_state=42):
        self.labels = labels  # classes names
        self.classes_num = len(labels)  # num of classes

        np.random.seed(
            random_state
        )  
        self.W = (
            np.random.randn(3073, self.classes_num) * 0.0001
        )  # generate random weights, reshape to add bias 
        self.batch_size = batch_size  # batch_size

    def fit(self, X_train, Y_train, learning_rate=1e-8):
        loss = 0.0  # обнуляем loss
        train_len = X_train.shape[0]  # num of examples
        indexes = list(range(train_len))  # indexes train_len
        random.shuffle(indexes)  

        for i in range(
            0, train_len, self.batch_size
        ):  
            idx = indexes[
                i : i + self.batch_size
            ]  # 
            X_batch = X_train[idx]  
            Y_batch = Y_train[idx]  

            X_batch = np.hstack(
                [X_batch, np.ones((X_batch.shape[0], 1))]
            )  # add bias

            loss_val, grad = self.loss(X_batch, Y_batch)  # loss and gradient
            self.W -= learning_rate * grad  # update weigths

            loss += loss_val  # loss sum
        return loss / (train_len)  # mean loss

    def loss(self, X, Y):
        current_batch_size = X.shape[0]  # batch_size
        loss = 0.0  
        dW = np.zeros(self.W.shape) 
        for i in range(current_batch_size):  
            scores = X[i].dot(
                self.W
            )  # vector of shape 10
            correct_class_score = scores[
                int(Y[i])
            ]  
            above_zero_loss_count = 0  
            for j in range(self.classes_num): 
                if j == Y[i]:  # predict class
                    continue
                margin = scores[j] - correct_class_score + 1  # loss
                if margin > 0:  
                    above_zero_loss_count += (
                        1  
                    )
                    loss += margin  # 
                    dW[:, j] += X[i]  # 
            dW[:, int(Y[i])] -= above_zero_loss_count * X[i]  
        loss /= current_batch_size  
        dW /= current_batch_size  
        return loss, dW

    def forward(self, X):
        X = np.append(X, 1)  # add 1 (bias)
        scores = X.dot(self.W)  
        return np.argmax(scores)  

In [ ]:

def validate(model, X_test, Y_test, noprint=False):
    correct = 0  
    for i, img in enumerate(X_test):  
        index = model.forward(img)  
        correct += (
            1 if index == Y_test[i] else 0
        )  
        if noprint is False:  
            if i > 0 and i % 1000 == 0:  
                print(
                    "Accuracy {:.3f}".format(correct / i)
                )  
    return correct / len(Y_test)  

In [ ]:
print("How learning quality depends of speed:")

for lr in [1e-2, 1e-8]:
    for bs in [256, 2048]:

        print("-" * 50, "\n", "learning_rate =", lr, "\tbatch_size =", bs)
        print()
        lc_model = LinearClassifier(labels_eng, batch_size=bs)

        best_accuracy = 0
        for epoch in range(10):
            loss = lc_model.fit(X_train, Y_train, learning_rate=lr)
            accuracy = validate(lc_model, X_test, Y_test, noprint=True)
            if best_accuracy < accuracy:
                best_accuracy = accuracy
                best_epoch = epoch
            print(f"Epoch {epoch} \tLoss: {loss}, \tAccuracy:{accuracy}")

        print()
        print(f"Best accuracy is {best_accuracy} in {best_epoch} epoch")

## Выбор шага обучения

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/update_weghts_values.png" width="450"> 

<!-- [Визуализация](https://docs.google.com/file/d/0Byvt-AfX75o1ZWxMRkxrUFJ2ZUE/preview)
 -->



Шаг обучения - некоторый коэффициент, как правило, небольшой, который не позволяет нам двигаться слишком быстро. У нас есть точка, в которую мы хотим попасть. Если мы сделаем слишком большой шаг, то мы ее перескочим (график справа), поэтому надо подобрать шаг, который не позволит ее перескочить, но в то же время, чтобы тот же процесс не шел слишком медленно (как на графике слева)

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/learning_rate_optimal_value.png" > 

 

Пока изменения loss функции достаточно большие, сам по себе градиент тоже большой. За счет этого можно двигаться быстро. Когда он будет уменьшаться, мы должны оказаться рядом с этим минимумом, и шаг, который мы выбрали, не должен мешать этому процессу. 

Бывают ситуации, когда шаг можно менять в самом процессе обучения, когда в начале обучения модели шаг большой, потом, по мере того, как она сходится, чтобы более четко найти минимум, шаг можно изменить вручную. Но изначально его нужно каким-то образом подобрать, и это зависит от данных и от самой модели. Это тоже гиперпараметр, связанный с обучением.

## Выбор размера батча



<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/define_size_of_batch.png" width="500"> 

Мы с самого начала говорили о выборках, некоторого количества примеров. В дальнейшем, мы будем называть их батчами. Батч (англ. *batch*) - это некоторое подмножество обучающей выборки фиксированного размера.

При этом было не очень понятно, чем они мотивированны. Точнее, мы мотивировали это тем, что у нас много данных, и мы не сможем их обработать все, и это правда. Даже если мы сможем загрузить все данные в память, нам нужно будет загрузить их и использовать при расчете, в том числе градиента. Это ещё более затратно. 

Зачем батчи нужны при рассчете loss? Если мы посчитаем loss по одному изображению, скорее всего он будет очень специфичен, и это движение, которое произойдет, будет направлено в сторону минимума, потому что по этому конкретному изображению мы улучшим показатели, что не отражает обобщения всех данных, поэтому мы используем батчи.

Нас в первую очередь интересует более общее направление, которое будет работать на большинстве наших данных. Поэтому если мы будем оптимизировать модели, исходя из одного элемента данных, путь будет витиеватый, и процесс будет происходить достаточно долго. Если же данные не помещаются в память, то **можно использовать батч того размера, который у нас есть**. Можно загрузить в память и считать по батчам градиент. В этом случае спуск будет более плавным, чем по одному изображению. 

Также при использовании всего датасета тоже есть свои минусы. **Не всегда загрузка всего датасета приводит к увеличению точности**. 

Подробнее: 
* [Методы оптимизации нейронных сетей
](https://habr.com/ru/post/318970/)
* [Обучение нейронной сети](https://alxmamaev.medium.com/deep-learning-на-пальцах-часть-1-341cfe021ef9)


## Понятие градиента

## Градиентный спуск

## Большие объемы данных

## Стохастический градиентный спуск

## Регуляризация весов



__Сложность модели__ (*model complexity*) &mdash; важный гиперпараметр. В частности, для линейной модели, сложность может быть представлена количеством параметров, для полиномиальных моделей &mdash; степенью полинома, для деревьев решений &mdash; глубиной дерева и т.д.

Сложность модели тесно связана с __ошибкой обобщения__ (_generalization error_). Ошибка обобщения отличается от ошибки обучения, измеряемой на тренировочных данных, тем, что позволяет оценить обобщающую способность модели, приобретенную в процессе обучения, давать точные ответы на неизвестных ей объектах. Cлишком простой модели не будет хватать мощности для обобщения сложной закономерности в данных, что приводит к большой ошибке обобщения, с другой стороны слишком сложная модель также приводит к большой ошибке обобщения за счет того, что в силу своей сложности модель начинает пытаться искать закономерности в шуме, добиваясь большей точности на тренировочных данных, теряя при этом часть обобщающей способности.

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/model_complexity.png" width="500"> 

Проиллюстрируем описанное явление на примере полиномиальной модели.

In [ ]:
x = np.linspace(0, 2*np.pi, 10)
y = np.sin(x) + np.random.normal(scale=0.25, size=len(x))
plt.scatter(x, y, s=50, facecolors='none', edgecolors='b', label='noisy data')

x_true = np.linspace(0, 2*np.pi, 200)
y_true = np.sin(x_true)
plt.plot(x_true, y_true, c='lime', label='ground truth')
plt.legend()
plt.show()


Попробуем аппроксимировать имеющуюся зависимость с помощью полиномиальной модели, используя шумные данные в качестве тренировочных данных:

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

x_train = x.reshape(-1,1)

fig = plt.figure(figsize=(12,6))

for i, degree in enumerate([0,1,3,9]):

    model = make_pipeline(PolynomialFeatures(degree), LinearRegression())

    model.fit(x_train, y)
    y_plot = model.predict(x_true.reshape(-1,1))

    fig.add_subplot(2,2,i+1)
    plt.plot(x_true, y_plot, c='red', label=f'M={degree}')
    plt.scatter(x, y, s=50, facecolors='none', edgecolors='b')
    plt.plot(x_true, y_true, c='lime')
    plt.legend()
plt.show()

Видно, что модель может переобучаться, подстраиваясь под тренировочную выборку. В полиноме степень, и как следствие количество весов &mdash; это гиперпараметр, который можно подбирать на кросс-валидации, однако, когда мы, таким образом, подбираем сложность модели мы налагаем довольно грубое ограничение на обобщающую способность модели в целом. Вместо этого более разумным было бы оставить модель сложной, но использовать некий ограничитель (__регуляризатор__), который будет заставлять модель отдавать предпочтение выбору более простого обобщения. 

In [ ]:
from sklearn.linear_model import Ridge

model = make_pipeline(PolynomialFeatures(9), LinearRegression())
model_ridge = make_pipeline(PolynomialFeatures(9), Ridge(alpha=0.1))

model.fit(x_train, y)
y_plot = model.predict(x_true.reshape(-1,1))

model_ridge.fit(x_train, y)
y_plot_ridge = model_ridge.predict(x_true.reshape(-1,1))

plt.plot(x_true, y_plot, c='red', label=f'M={degree}')
plt.plot(x_true, y_plot_ridge, c='black', label=f'M={degree}, alpha=0.1')
plt.scatter(x, y, s=50, facecolors='none', edgecolors='b')
plt.plot(x_true, y_true, c='lime', label='ground truth')
plt.legend()
plt.show()

poly_coef = model[1].coef_

eq = f'y = {round(poly_coef[0], 2)}+{round(poly_coef[1], 2)}*x'
for i in range(2, 10):
    eq += f'+{round(poly_coef[i], 2)}*x^{i}'
    
print('Without regularization: ', eq)

poly_coef = model_ridge[1].coef_

eq = f'y = {round(poly_coef[0], 2)}+{round(poly_coef[1], 2)}*x'
for i in range(2, 10):
    eq += f'+{round(poly_coef[i], 2)}*x^{i}'

print('With regularization: ', eq)

Видно, что одним из "симптомов" переобучения являются аномально большие веса. Модель Ridge Regression, показанная в примере выше, использует L2 регуляризуцию для борьбы с этим явлением.




<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/l2_regularization.jpg" width="700"> 


L2 Regularization = weights decay

Идея состоит в том, что мы можем наложить некоторое требование на сами веса. Дело в том, что можно получить один и тот же выход модели при разных весах (выход модели соответствует умножению весов на $x$), при разных $w$ выход может быть идентичен.

Эти параметры задают некоторую аппроксимацию нашей целевой функции, аппроксимировать функцию можно двумя способами:
1. Использовать все имеющиеся данные и провести ее строго через все точки, которые нам известны; 
2. Использовать более простую функцию: (в данном случае линейную), которая не попадет точно во все данные, но зато будет соответствовать некоторым общим закономерностям, которые у них есть. 

Характерной чертой переобучения является второй сценарий, и сопровождается он, как правило, большими весами. Введение L2-регуляризации приводит к тому, что большие веса штрафуются и предпочтение отдается решениям, использующим малые значения весов. 

Модель может попробовать схитрить и по-другому - использовать все веса, все признаки, даже незначимые, но с маленькими коэффициентами. С этим L2-loss поможет хуже, так как он не сильнее штрафует мелкие веса. Результатов его применения - малые значения весов, которые использует модель

В этом случае на помощь приходит L1-loss, который штрафует вес за сам факт отличия его от нуля. Но и штрафует он все веса одинаково. Результат его применения - малое число весов, которые использует модель в принципе. 



<img src="https://edunet.kea.su/repo/EduNet-content/L02/img_license/l1_and_l2_regularization.gif" alt="alttext" width="550"/>


Это лоссы можно комбинировать - получится Elastic Net




$\lambda=$ regularization strength (hyperparameter)

$L(W)=\underbrace{\frac1N\sum_{i=1}^NL_i(f(x_i,W),y_i)}_{\textbf{Data loss} }+\underbrace {\lambda R(W)}_{\textbf{Regularization}}$

Берем сумму всех весов по всей матрице $w$, и добавляем ее к loss. Соответственно, чем больше будет эта сумма, тем больше будет суммарный loss. 


В дальнейшем проблема с переобучением будет вставать довольно часто. Методов регуляризации модели существует достаточно много. Этот — один из базовых, который будет использоваться практически во всех оптимизаторах, с которыми познакомимся позже.

# Логистическая регрессия

## Логистическая регрессия для бинарной классификации

## Решение с помощью градиентного спуска

## Мультиклассовая классификация (объект может принадлежать к нескольким классам)

## Кросс-энтропия, softmax и логистическая регрессия для мультиклассовой постановки

## Переход к вероятностям

**Softmax**

[Видео от StatQuest, которое объясняет Softmax](https://www.youtube.com/watch?v=KpKog-L9veg)

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/scores_to_probability.png" width="750">

Перейти к вероятностям мы сможем, проведя с весами некоторые не очень сложные математические преобразования. 

На слайде выше показано, почему выходы модели часто называют [logit’ами](https://en.wikipedia.org/wiki/Logit). Если предположить, что у нас есть некая вероятность, от которой мы берем такую функцию (logit), то она может принимать значения от нуля до бесконечности. Мы можем считать, что выходы модели - это logit’ы. 




Например, мы могли бы просто взять индекс массива, в котором значение (logit) максимально. Предположим, что наша сеть выдала следующие значения:

In [ ]:
import numpy as np

logits = [
          5.1, # cat
          3.2, # car
          -1.7, # frog
]

Тогда, чтобы узнать какой класс наша сеть предсказала, мы могли бы просто взять `argmax` от наших `logits`

In [ ]:
print('Predicted class = %i (Cat)' % (np.argmax(logits)))

Но от argmax нельзя посчитать градиент, так как производная от константы равна 0. Соответственно, если бы мы вставили производную от argmax в градиентный спуск, мы бы получили везде нули, и соответственно, наша сеть бы вообще ничему не научилась

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(np.arange(3), [1,0,0], color='red', s=50)
plt.show()

А мы бы хотели получить не logit’ы, а настоящую вероятность на выходе модели. Да еще и таким образом, что бы от наших вероятностей можно было бы посчитать градиент. Для этого мы можем применить к нашим логитам функцию **Softmax**

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/linear_classifier_softmax.jpg" width="1300">

Мы можем провести над логитами операцию экспоненцирования, то есть число Эйлера (2.71828) **возвести в степень**, соответствующую этому выходу. В результате, мы получим вектор, гарантировано неотрицательных чисел (потому что мы ввели неположительное число в степень, пускай даже отрицательную, то есть выходы могут быть маленькие, но всегда положительные). 



Дальше, чтобы можно было интерпретировать эти числа как вероятности, их сумма должна быть равна единице. Мы должны их нормализовать, то есть **поделить на сумму**. Это преобразование называется **Softmax функцией**.

**Получаются вероятности**, то есть числа, которые можно интерпретировать таким образом.

$\text{Softmax}_\text{кошка} = \frac{e^{5.1}}{e^{5.1} + e^{3.2} + e^{-1.7}}$

In [ ]:
def softmax(logits):
    return np.exp(logits)/np.sum(np.exp(logits))

print(softmax(logits))
print('Sum = %.2f' % np.sum(softmax(logits)))

Можно обратить внимание, что Softmax, никоим образом не поменял порядок значений. Самому большому logit'у соответствует самая большая вероятность, а самому маленькому, соответственно самая маленькая

Посмотрим на графиках. Возьмем массив случайных логитов и применим к ним softmax

In [ ]:
rand_logits = np.linspace(-1,1,50)
fig,ax = plt.subplots(ncols=2)

ax[0].plot(np.arange(50), rand_logits)
ax[0].set_title('Logits')
ax[1].plot(np.arange(50), softmax(rand_logits))
ax[1].set_title('Softmax')
plt.show()

**Cross-entropy / log loss**

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/cross_entropy_plot_loss_with_probability.png" width="800">


Теперь, с использованием такого вектора можно определить другую loss функцию. Если не вникать в детали, можно **взять от нее логарифм**. Тогда loss от такого выхода будет выглядеть вот таким образом:

$ L_i =  -log(\frac{e^{s_{y_i}}}{\sum_j e^{s_j}})$

как нормализованный выход от верного класса на сумму остальных, к которому добавили минус. 

Получится график loss (слева). Его плюс заключается в том, что у него нет участка с плато, практически по всей длине функция получается гладкой, с хорошими мощными производными. Когда loss большой, производная тоже большая, и за счет этого можно быстро обучать модель. Для кусочно-линейной функции потерь она же равна константе.

До тех пор, пока модель не будет работать с точностью 100%, то есть пока все остальные выходы не станут нулевыми, мы можем продолжить обучение, даже если у нас нет явной регуляризации.

Осталось выяснить, почему такая простая на вид функция **называется так сложно** - Кросс-энтропия. 



## Определение Кросс-энтропии




Начнем с простого примера, пусть у нас есть 10 точек со следующими значениями признака x: 

`x = [-2.2, -1.4, -0.8, 0.2, 0.4, 0.8, 1.2, 2.2, 2.9, 4.6]`

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/cross_entropy_ten_dots.png">

Пусть наши точки принадлежат двум классам: зеленый и красный:

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/cross_entropy_two_classes.png">

Перед нами простая задача классификации: по признаку x предсказать класс наших точек. Мы можем переформулировать задачу как нахождение вероятности того, что точка зеленая или красная. В идеальной ситуации для зеленой точки вероятность того, что она зеленая равна 1, в то же время вероятность того, что красная точка &mdash; зеленая должна быть равна 0. 

Вероятности, которые выдает, обучаемая нами модель, зачастую далеки от идеальной ситуации. В нашем примере сравнить насколько сильно предсказанная вероятность класса отличается от вероятности, выдаваемой "идеальной моделью" можно за счет __бинарной кросс-энтропии__ частного случая кросс-энропии.

$$H_p(q)=-\frac{1}{N}\sum^N_{i=1}y_i\cdot log(p(y_i))+(1-y_i)\cdot log(1-p(y_i))$$

где $y$ &mdash; метка класса (1 для зеленого, 0 для красного), которую можно также интерпретировать как вероятность, предсказанную "идеальной моделью", $p(y)$ &mdash; вероятность того, что точка зеленая, предсказываемая оцениваемой моделью.

Какое же отношение энтропия имеет к этой формуле? Давайте углубимся в детали.

Поскольку $y$ представляет метку классов точек, то его распределение $q(y)$ выглядит следующим образом:

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/cross_entropy_distribution.png">

__Энтропия__ &mdash; мера неуверенности, связанная с распределением $q(y)$. Какова была бы мера неуверенности распределения $q(y)$ если бы все точки были зелеными? Так как у нас бы не было сомнений насчет цвета точки (он всегда зеленый), значение энтропии было бы 0. Теперь представим другую ситуацию, пусть у нас поровну точек зеленого и красного цвета. Для нас это наихудшая ситуация, поскольку попытка определить цвет точки, по сути, представляет случайное угадывание. В этом случае энтропия вычисляется по формуле Хартли:

$$H(q)=log(2)$$

Мы рассмотрели два крайних случая, но как быть с промежуточными ситуациями? Для этих случаев мы можем использовать формулу Шеннона:

$$H(q)=-\sum^C_{c=1}q(y_c)\cdot log(q(y_c))$$

где C &mdash; количество классов. Нетрудно заметить, что формула Хартли является частным случаем формулы Шеннона.

Таким образом, зная истинное распределение случайной величины, мы можем рассчитать его энтропию. А что будет если мы попытаемся аппроксимировать истинное распределение $q(y)$ некоторым другим распределением $p(y)$? Допустим, что наши цветные точки подчиняются этому распределению $p(y)$, также мы знаем, что исходят они из неизвестного нам истинного распределения $q(y)$, если мы посчитаем следующую энтропию, это и будет __кросс-энтропия__:

$$H_p(q)=-\sum^C_{c=1}q(y_c)\cdot log(p(y_c))$$

Если окажется, что распределения $p(y)$ и $q(y)$ совпадают, в этом случае энтропия $H(q)$ и кросс-энтропия $H_p(q)$ также будут совпадать. Однако в реальности такое случается редко и кросс-энтропия бывает больше энтропии истинного распределения

$$H_p(q)-H(q)\geq0$$

Разница между кросс-энтропией и энтропией называется __дивергенцией Кульбака-Лейблера__, которая является мерой различия между двумя распределениями:

$$D_{KL}(q||p)=H_p(q)-H(q)=\sum^C_{c=1}q(y_c)\cdot [log(q(y_c))-log(p(y_c))]$$

Это значит, что чем ближе $p(y)$ к $q(y)$, тем меньше будет значение дивергенции Кульбака-Лейблера и, следовательно, меньше значение кросс-энтропии.

Таким образом, мы хотим добиться, чтобы модель, которую мы оцениваем, порождала $p(y)$ близкое к $q(y)$. Для этих целей мы стремимся __минимизировать кросс-энтропию__.



<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/define_cross_entropy_kulbak_shennon.png" >

Если мы рассматриваем выходы нашей модели как вероятности, то мы можем их сравнивать. У нас есть номер правильного класса. Соответственно, можно сказать, что вероятность этого класса - единица, а всех остальных 0, и получить, таким образом, вероятностное распределение. Выход модели тоже можно интерпретировать как вероятности (тоже можно получить вероятностное распределение). И для работы с этими распределением есть некоторый математический аппарат, который основан на понятии энтропии, который ввел Клод Шеннон.


<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/efficient_way_to_send_signals.png" >


Идея в следующем: у нас есть некоторые данные, которые мы передаем по каналу связи. Например, у нас есть метеостанция, которая сообщает прогноз погоды. Допустим, она может передавать 8 вариантов прогноза. Мы в каждый момент времени получаем от нее сообщение. Предположим, что потребуется 3 бит, чтобы передавать это сообщение. 

Допустим, мы знаем некую вероятность, с которой в нашем регионе может наступить хорошая либо плохая погода. То есть мы знаем вероятностное распределение, по которому у нас, допустим, солнечное место, где почти не бывает бури. Если мы знаем об этом, мы можем сэкономить на передаче информации. Мы можем закодировать наиболее вероятные сообщения двумя битами, причем придумать такие коды, чтобы они почти не пересекались. Они будут начинаться с нуля и передавать два бита вместо трех. Таким образом, можно сэкономить на передачи информации. То есть идея такая: **если мы знаем, что события маловероятны, мы можем кодировать их более длинной цепочкой, а более вероятные события - более короткими цепочками**. В этом случае в среднем количество информации, которое можно передать, сократится.

**Формула Шеннона позволяет посчитать, насколько сильно мы можем сэкономить для конкретного вероятностного распределения**. То есть если подставить эти данные в формулу, то получим 2,23.

На практике реализовать это, используя биты, возможно, не выйдет, но это свойство данного вероятностного распределения можно посчитать. Оно несёт в себе некоторое количество полезной информации, соответствующее этой энтропии.

**Что же такое кросс-энтропия?**

Давайте представим, что мы каким-то образом начали кодировать эту информацию, прошили погодную станцию так, что она может эту кодированную информацию передавать, а потом перенесли в другой регион, где ситуация противоположная. И мы кодируем события, которые стали частыми, длинными цепочками, а маловероятные - короткой, и у нас в этом случае явно передается больше информации, чем мы могли бы, если бы сделали все банально. 

Кросс-энтропия позволяет посчитать, насколько большая будет потеря в данном случае. То есть это некий **способ сравнить между собой вероятностные распределения**.

Что, собственно говоря, отображено в формуле, потому что мы считаем кросс-энтропию по нашему вектору $P$ и $Q$. Вектор $P$ состоит из нулей и единиц, соответственно, во всех случаях, кроме одного, это выражение будет нулевым, кроме одного случая с единицей. Случай с единицей соответствует нашему правильному классу. Поэтому сумма исчезает, остаются логарифм и вектор, а вероятность для правильного класса мы считаем как Soft max. Отсюда название кросс-энтропия.  


(В теории информации кросс-энтропия между двумя распределениями вероятностей $p$ и $q$  по одному и тому же базовому набору событий измеряет среднее число битов, необходимых для идентификации события, взятого из набора, если схема кодирования, используемая для набора, оптимизирована для оценочного распределения вероятностей $q$, а не для истинного распределения $p$.)

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/logits_to_scores_to_probabilitys.png" width="900">

Аналогичное обоснование: есть понятие дивергенция, которая позволяет оценить расхождение между вероятностными распределениями, которая нам здесь и нужна. Есть дивергенция **Кульбака-Лейблера**, которая выражается через кросс-энтропию, а энтропия от нашего вектора нулевая. Поэтому фактически здесь кросс-энтропия равна дивергенции, которая показывает, насколько не похожи два распределения. 


<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/softmax_formula.jpg" width="800">


Эта функция очень популярна, она используется при обучении реальных моделей на практике. Поскольку внутри находится Softmax, часто ее называют Softmax-классификатором, что, строго говоря, не совсем верно. Здесь функция потерь - кросс-энтропия, Softmax же просто используется внутри нее.
